In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression_linear(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
def NN_model_structure_regression_relu(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('relu'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [20]:
def NN_model_structure_regression_sigmoid(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('sigmoid'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [21]:
def NN_model_structure_regression_tanh(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('tanh'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [22]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [23]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [24]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_linear(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 418us/step - 

1052/1052 [==============================] - 0s 99us/step - loss: 21.6102 - val_loss: 25.6163
Epoch 66/1000
1052/1052 [==============================] - 0s 85us/step - loss: 21.6835 - val_loss: 26.8271
Epoch 67/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.0123 - val_loss: 25.6005
Epoch 68/1000
1052/1052 [==============================] - 0s 88us/step - loss: 22.2203 - val_loss: 25.6938
Epoch 69/1000
1052/1052 [==============================] - 0s 92us/step - loss: 22.0446 - val_loss: 26.0751
Epoch 70/1000
1052/1052 [==============================] - 0s 89us/step - loss: 21.8529 - val_loss: 28.4044
Epoch 71/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.8004 - val_loss: 26.4922
Epoch 72/1000
1052/1052 [==============================] - 0s 91us/step - loss: 22.2209 - val_loss: 26.3785
Epoch 73/1000
1052/1052 [==============================] - 0s 85us/step - loss: 22.0070 - val_loss: 29.0429
Epoch 74/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 98us/step - loss: 22.9685 - val_loss: 27.4244
Epoch 141/1000
1052/1052 [==============================] - 0s 88us/step - loss: 23.6275 - val_loss: 27.1499
Epoch 142/1000
1052/1052 [==============================] - 0s 84us/step - loss: 21.9534 - val_loss: 26.4189
Epoch 143/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.7800 - val_loss: 25.8620
Epoch 144/1000
1052/1052 [==============================] - 0s 83us/step - loss: 22.6024 - val_loss: 26.0463
Epoch 145/1000
1052/1052 [==============================] - 0s 89us/step - loss: 22.3630 - val_loss: 26.8442
Epoch 146/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.4092 - val_loss: 25.6580
Epoch 147/1000
1052/1052 [==============================] - 0s 88us/step - loss: 22.5018 - val_loss: 26.6457
Epoch 148/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.0331 - val_loss: 25.2813
Epoch 149/1000
1052/1052 [========

1052/1052 [==============================] - 0s 85us/step - loss: 20.2159 - val_loss: 24.8436
Epoch 216/1000
1052/1052 [==============================] - 0s 90us/step - loss: 19.2494 - val_loss: 24.0095
Epoch 217/1000
1052/1052 [==============================] - 0s 93us/step - loss: 19.4951 - val_loss: 23.5365
Epoch 218/1000
1052/1052 [==============================] - 0s 90us/step - loss: 20.0485 - val_loss: 23.7629
Epoch 219/1000
1052/1052 [==============================] - 0s 88us/step - loss: 18.7663 - val_loss: 23.3365
Epoch 220/1000
1052/1052 [==============================] - 0s 91us/step - loss: 19.9557 - val_loss: 24.1125
Epoch 221/1000
1052/1052 [==============================] - 0s 92us/step - loss: 18.4550 - val_loss: 22.4133
Epoch 222/1000
1052/1052 [==============================] - 0s 91us/step - loss: 18.2295 - val_loss: 23.5066
Epoch 223/1000
1052/1052 [==============================] - 0s 87us/step - loss: 18.6564 - val_loss: 21.7443
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 13.5423 - val_loss: 18.9510
Epoch 291/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.0356 - val_loss: 16.8021
Epoch 292/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.3412 - val_loss: 17.5294
Epoch 293/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.4205 - val_loss: 16.7293
Epoch 294/1000
1052/1052 [==============================] - 0s 85us/step - loss: 12.7356 - val_loss: 15.6536
Epoch 295/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.8000 - val_loss: 18.1133
Epoch 296/1000
1052/1052 [==============================] - 0s 91us/step - loss: 13.2034 - val_loss: 15.1849
Epoch 297/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.5440 - val_loss: 15.8667
Epoch 298/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.6506 - val_loss: 15.9025
Epoch 299/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 90us/step - loss: 8.3568 - val_loss: 10.4212
Epoch 746/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6808 - val_loss: 9.0661
Epoch 747/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.0205 - val_loss: 9.7801
Epoch 748/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.4747 - val_loss: 10.0842
Epoch 749/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.4284 - val_loss: 11.9391
Epoch 750/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.6211 - val_loss: 10.2931
Epoch 751/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3541 - val_loss: 9.5445
Epoch 752/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3546 - val_loss: 9.2796
Epoch 753/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.1659 - val_loss: 9.0317
Epoch 754/1000
1052/1052 [======================

1052/1052 [==============================] - 0s 89us/step - loss: 8.1688 - val_loss: 9.4526
Epoch 822/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.9840 - val_loss: 9.4178
Epoch 823/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.1220 - val_loss: 9.3597
Epoch 824/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.4813 - val_loss: 9.5743
Epoch 825/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3106 - val_loss: 9.2162
Epoch 826/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.1029 - val_loss: 9.2140
Epoch 827/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.4176 - val_loss: 9.7894
Epoch 828/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.4505 - val_loss: 9.5401
Epoch 829/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3123 - val_loss: 9.1489
Epoch 830/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 99us/step - loss: 8.3766 - val_loss: 9.5189
Epoch 898/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.7832 - val_loss: 10.5760
Epoch 899/1000
1052/1052 [==============================] - 0s 99us/step - loss: 8.5605 - val_loss: 10.4051
Epoch 900/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.4854 - val_loss: 9.9773
Epoch 901/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.3411 - val_loss: 9.5852
Epoch 902/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.0575 - val_loss: 9.0697
Epoch 903/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.1482 - val_loss: 11.6769
Epoch 904/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.3229 - val_loss: 10.2671
Epoch 905/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3974 - val_loss: 9.2429
Epoch 906/1000
1052/1052 [=====================

1052/1052 [==============================] - 0s 89us/step - loss: 8.0918 - val_loss: 9.2804
Epoch 974/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.2160 - val_loss: 9.8550
Epoch 975/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2673 - val_loss: 9.4276
Epoch 976/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.4609 - val_loss: 9.1479
Epoch 977/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2409 - val_loss: 10.3247
Epoch 978/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.7467 - val_loss: 10.1517
Epoch 979/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1482 - val_loss: 8.9801
Epoch 980/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.7771 - val_loss: 10.5575
Epoch 981/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.2054 - val_loss: 9.8168
Epoch 982/1000
1052/1052 [=======================

[array([[ 0.7087425 ,  0.1404251 , -3.193     , -1.9082919 , -3.7617931 ],
        [ 1.4338864 ,  0.18494618,  0.07939274,  0.4951963 ,  0.19296935],
        [ 1.6166356 ,  0.29391664, -0.00454639,  0.38734654, -0.71108216],
        [-0.24788852, -0.12054072,  0.06733918,  0.01921353,  0.14085282],
        [-0.31918317,  0.18439576, -2.3774862 , -0.31030443, -0.31814748]],
       dtype=float32),
 array([ 0.739141 , -1.3315203, -4.6797523, -1.2957648, -4.72295  ],
       dtype=float32),
 array([[ 0.09057587,  0.22450846, -0.24725057,  0.06204393,  0.6878021 ,
         -0.6110786 , -0.06546178, -0.88051337, -0.7157743 ,  0.39152974],
        [-1.1620767 ,  1.0693033 ,  2.1112447 ,  1.283406  ,  1.0418489 ,
         -1.4466256 , -1.6035215 , -2.1358845 , -2.1962378 ,  1.3185877 ],
        [-1.6225098 ,  2.082052  ,  1.8690599 ,  2.0784264 ,  1.1625152 ,
         -1.5635835 , -1.3624095 , -1.7895293 , -1.8424351 ,  1.3716922 ],
        [ 0.3014366 , -0.9622789 , -0.2595032 , -0.88682103, -

In [25]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_linear_4th.h5')

In [26]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_relu(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 199us/step - l

1052/1052 [==============================] - 0s 93us/step - loss: 14.9724 - val_loss: 17.1008
Epoch 68/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.8595 - val_loss: 17.9312
Epoch 69/1000
1052/1052 [==============================] - 0s 92us/step - loss: 15.1943 - val_loss: 17.3325
Epoch 70/1000
1052/1052 [==============================] - 0s 91us/step - loss: 14.5259 - val_loss: 16.2603
Epoch 71/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.0577 - val_loss: 16.7346
Epoch 72/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.9345 - val_loss: 16.9591
Epoch 73/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.9974 - val_loss: 15.6633
Epoch 74/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.8160 - val_loss: 17.0072
Epoch 75/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.1130 - val_loss: 19.9702
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 91us/step - loss: 12.6208 - val_loss: 14.1552
Epoch 143/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.0916 - val_loss: 13.9288
Epoch 144/1000
1052/1052 [==============================] - 0s 95us/step - loss: 12.4290 - val_loss: 14.9225
Epoch 145/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.0300 - val_loss: 13.8967
Epoch 146/1000
1052/1052 [==============================] - 0s 93us/step - loss: 12.0066 - val_loss: 14.8923
Epoch 147/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.1960 - val_loss: 15.5275
Epoch 148/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.9258 - val_loss: 14.1855
Epoch 149/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.8511 - val_loss: 14.2220
Epoch 150/1000
1052/1052 [==============================] - 0s 90us/step - loss: 12.4805 - val_loss: 13.7983
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 88us/step - loss: 11.7717 - val_loss: 15.3542
Epoch 218/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.2094 - val_loss: 16.0923
Epoch 219/1000
1052/1052 [==============================] - 0s 95us/step - loss: 12.2644 - val_loss: 14.6878
Epoch 220/1000
1052/1052 [==============================] - 0s 92us/step - loss: 12.8987 - val_loss: 15.6299
Epoch 221/1000
1052/1052 [==============================] - 0s 101us/step - loss: 12.3225 - val_loss: 15.5224
Epoch 222/1000
1052/1052 [==============================] - 0s 95us/step - loss: 12.0139 - val_loss: 14.6748
Epoch 223/1000
1052/1052 [==============================] - 0s 96us/step - loss: 12.0791 - val_loss: 14.9360
Epoch 224/1000
1052/1052 [==============================] - 0s 97us/step - loss: 11.7011 - val_loss: 14.1710
Epoch 225/1000
1052/1052 [==============================] - 0s 94us/step - loss: 12.0840 - val_loss: 14.7601
Epoch 226/1000
1052/1052 [=======

1052/1052 [==============================] - 0s 94us/step - loss: 11.6632 - val_loss: 14.2834
Epoch 293/1000
1052/1052 [==============================] - 0s 99us/step - loss: 12.0728 - val_loss: 15.1785
Epoch 294/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.3737 - val_loss: 14.7496
Epoch 295/1000
1052/1052 [==============================] - 0s 86us/step - loss: 12.1978 - val_loss: 17.8168
Epoch 296/1000
1052/1052 [==============================] - 0s 92us/step - loss: 12.4388 - val_loss: 14.2765
Epoch 297/1000
1052/1052 [==============================] - 0s 92us/step - loss: 12.2724 - val_loss: 14.6713
Epoch 298/1000
1052/1052 [==============================] - 0s 92us/step - loss: 11.7118 - val_loss: 15.8262
Epoch 299/1000
1052/1052 [==============================] - 0s 86us/step - loss: 12.6839 - val_loss: 14.0656
Epoch 300/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.9050 - val_loss: 14.3071
Epoch 301/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 84us/step - loss: 11.8050 - val_loss: 14.1631
Epoch 743/1000
1052/1052 [==============================] - 0s 84us/step - loss: 12.1146 - val_loss: 14.2022
Epoch 744/1000
1052/1052 [==============================] - 0s 84us/step - loss: 11.6969 - val_loss: 13.8061
Epoch 745/1000
1052/1052 [==============================] - 0s 83us/step - loss: 11.6109 - val_loss: 15.4448
Epoch 746/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.3434 - val_loss: 14.7322
Epoch 747/1000
1052/1052 [==============================] - 0s 91us/step - loss: 11.8544 - val_loss: 14.4352
Epoch 748/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.1207 - val_loss: 13.6761
Epoch 749/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.2597 - val_loss: 13.3124
Epoch 750/1000
1052/1052 [==============================] - 0s 83us/step - loss: 11.1852 - val_loss: 14.7308
Epoch 751/1000
1052/1052 [========

1052/1052 [==============================] - 0s 84us/step - loss: 11.1065 - val_loss: 13.0433
Epoch 818/1000
1052/1052 [==============================] - 0s 83us/step - loss: 11.2010 - val_loss: 13.6659
Epoch 819/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.6026 - val_loss: 14.9567
Epoch 820/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.0327 - val_loss: 14.6176
Epoch 821/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.8462 - val_loss: 15.1965
Epoch 822/1000
1052/1052 [==============================] - 0s 84us/step - loss: 11.4991 - val_loss: 14.3393
Epoch 823/1000
1052/1052 [==============================] - 0s 85us/step - loss: 11.8734 - val_loss: 14.7174
Epoch 824/1000
1052/1052 [==============================] - 0s 85us/step - loss: 11.5984 - val_loss: 16.8286
Epoch 825/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.7320 - val_loss: 14.4154
Epoch 826/1000
1052/1052 [========

1052/1052 [==============================] - 0s 103us/step - loss: 11.8150 - val_loss: 13.9382
Epoch 893/1000
1052/1052 [==============================] - 0s 103us/step - loss: 12.1843 - val_loss: 13.5576
Epoch 894/1000
1052/1052 [==============================] - 0s 112us/step - loss: 11.6417 - val_loss: 14.0283
Epoch 895/1000
1052/1052 [==============================] - 0s 101us/step - loss: 11.6433 - val_loss: 13.9761
Epoch 896/1000
1052/1052 [==============================] - 0s 95us/step - loss: 11.2276 - val_loss: 14.4472
Epoch 897/1000
1052/1052 [==============================] - 0s 113us/step - loss: 11.5458 - val_loss: 14.3717
Epoch 898/1000
1052/1052 [==============================] - 0s 104us/step - loss: 11.6625 - val_loss: 15.4203
Epoch 899/1000
1052/1052 [==============================] - 0s 94us/step - loss: 11.8134 - val_loss: 13.5183
Epoch 900/1000
1052/1052 [==============================] - 0s 98us/step - loss: 11.3131 - val_loss: 14.6644
Epoch 901/1000
1052/1052 [==

1052/1052 [==============================] - 0s 87us/step - loss: 11.5590 - val_loss: 13.6896
Epoch 968/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.4907 - val_loss: 14.2057
Epoch 969/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.1396 - val_loss: 13.5587
Epoch 970/1000
1052/1052 [==============================] - 0s 92us/step - loss: 11.7622 - val_loss: 14.3505
Epoch 971/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.4928 - val_loss: 15.0824
Epoch 972/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.4836 - val_loss: 13.6322
Epoch 973/1000
1052/1052 [==============================] - 0s 84us/step - loss: 11.3970 - val_loss: 13.4072
Epoch 974/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.5201 - val_loss: 13.4287
Epoch 975/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.2490 - val_loss: 14.3376
Epoch 976/1000
1052/1052 [========

[array([[-0.31571296, -0.38110504, -3.769983  ,  0.11913731, -1.7195567 ],
        [-0.23514554,  0.02222654, -0.06436571, -0.25299507,  0.8710252 ],
        [ 2.206672  ,  0.5852924 , -0.9175721 , -2.1951203 , -0.03383337],
        [ 0.07774983,  0.03995496,  0.18881723,  0.05850926,  0.17726563],
        [ 0.4514462 , -1.028287  , -0.50851274, -0.19075231,  0.96649754]],
       dtype=float32),
 array([ 2.6104069,  3.5114892, -2.7942383,  3.6506455,  0.8097739],
       dtype=float32),
 array([[-0.3438415 ,  1.1507928 , -0.5571401 ,  0.03497237,  0.8969057 ,
         -1.0122128 ,  0.989814  , -0.23544563,  1.2580944 , -0.1907548 ],
        [-0.7738095 ,  0.5596551 , -0.07651436,  1.0835688 , -0.01368968,
         -0.39661095,  1.4969416 , -0.22487035,  1.5679551 , -0.11425459],
        [-0.4324529 , -1.5551146 , -0.41895422, -1.6259886 , -1.133219  ,
         -0.6088671 , -1.2403544 , -0.8458967 , -1.5708776 , -0.42115033],
        [-0.26141056,  1.2381513 , -0.32299986,  0.6694634 ,  

In [27]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_relu_4th.h5')

In [28]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_sigmoid(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 246us/step - l

1052/1052 [==============================] - 0s 91us/step - loss: 17.7908 - val_loss: 21.1798
Epoch 67/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.2314 - val_loss: 20.9720
Epoch 68/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.8298 - val_loss: 20.8723
Epoch 69/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.5506 - val_loss: 20.4488
Epoch 70/1000
1052/1052 [==============================] - 0s 99us/step - loss: 16.1849 - val_loss: 20.4130
Epoch 71/1000
1052/1052 [==============================] - 0s 101us/step - loss: 16.0672 - val_loss: 20.2290
Epoch 72/1000
1052/1052 [==============================] - 0s 90us/step - loss: 15.6768 - val_loss: 19.9998
Epoch 73/1000
1052/1052 [==============================] - 0s 91us/step - loss: 15.5176 - val_loss: 19.7937
Epoch 74/1000
1052/1052 [==============================] - 0s 89us/step - loss: 15.3073 - val_loss: 19.6714
Epoch 75/1000
1052/1052 [================

1052/1052 [==============================] - 0s 87us/step - loss: 13.1853 - val_loss: 17.8115
Epoch 142/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.0716 - val_loss: 17.7737
Epoch 143/1000
1052/1052 [==============================] - 0s 91us/step - loss: 13.1890 - val_loss: 17.7326
Epoch 144/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.2173 - val_loss: 17.8727
Epoch 145/1000
1052/1052 [==============================] - 0s 97us/step - loss: 13.1556 - val_loss: 18.1151
Epoch 146/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.0798 - val_loss: 17.6284
Epoch 147/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.0067 - val_loss: 18.0156
Epoch 148/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.1078 - val_loss: 17.6632
Epoch 149/1000
1052/1052 [==============================] - 0s 92us/step - loss: 13.0298 - val_loss: 17.5938
Epoch 150/1000
1052/1052 [========

1052/1052 [==============================] - 0s 90us/step - loss: 12.5801 - val_loss: 17.3778
Epoch 217/1000
1052/1052 [==============================] - 0s 100us/step - loss: 12.4965 - val_loss: 17.5076
Epoch 218/1000
1052/1052 [==============================] - 0s 90us/step - loss: 12.4226 - val_loss: 17.5552
Epoch 219/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.4586 - val_loss: 17.5308
Epoch 220/1000
1052/1052 [==============================] - 0s 81us/step - loss: 12.4575 - val_loss: 17.5970
Epoch 221/1000
1052/1052 [==============================] - 0s 86us/step - loss: 12.3955 - val_loss: 17.4683
Epoch 222/1000
1052/1052 [==============================] - 0s 86us/step - loss: 12.4855 - val_loss: 17.5269
Epoch 223/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.4177 - val_loss: 17.7350
Epoch 224/1000
1052/1052 [==============================] - 0s 84us/step - loss: 12.4589 - val_loss: 17.8467
Epoch 225/1000
1052/1052 [=======

1052/1052 [==============================] - 0s 88us/step - loss: 12.0968 - val_loss: 17.6767
Epoch 292/1000
1052/1052 [==============================] - 0s 85us/step - loss: 12.2351 - val_loss: 17.5918
Epoch 293/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.0757 - val_loss: 17.5527
Epoch 294/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.1420 - val_loss: 17.7139
Epoch 295/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.9874 - val_loss: 17.4523
Epoch 296/1000
1052/1052 [==============================] - 0s 85us/step - loss: 12.1152 - val_loss: 17.5758
Epoch 297/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.1831 - val_loss: 17.4688
Epoch 298/1000
1052/1052 [==============================] - 0s 85us/step - loss: 12.0018 - val_loss: 17.3204
Epoch 299/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.9857 - val_loss: 17.4127
Epoch 300/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 90us/step - loss: 6.1056 - val_loss: 7.5184
Epoch 745/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.1034 - val_loss: 7.8089
Epoch 746/1000
1052/1052 [==============================] - 0s 91us/step - loss: 6.2356 - val_loss: 7.4426
Epoch 747/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.0912 - val_loss: 7.6336
Epoch 748/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.1722 - val_loss: 7.8355
Epoch 749/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.0228 - val_loss: 7.5741
Epoch 750/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.1379 - val_loss: 7.3352
Epoch 751/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.1346 - val_loss: 7.2654
Epoch 752/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.0567 - val_loss: 7.2609
Epoch 753/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 91us/step - loss: 5.9852 - val_loss: 6.8195
Epoch 821/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.9261 - val_loss: 7.0333
Epoch 822/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.8985 - val_loss: 7.1127
Epoch 823/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.8746 - val_loss: 7.3572
Epoch 824/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.0234 - val_loss: 7.0143
Epoch 825/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.9851 - val_loss: 6.9856
Epoch 826/1000
1052/1052 [==============================] - 0s 83us/step - loss: 5.8521 - val_loss: 6.7850
Epoch 827/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.7815 - val_loss: 6.7968
Epoch 828/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.9119 - val_loss: 6.8642
Epoch 829/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 91us/step - loss: 5.7703 - val_loss: 6.8950
Epoch 897/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.6827 - val_loss: 6.8405
Epoch 898/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.7579 - val_loss: 7.2287
Epoch 899/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.7678 - val_loss: 7.1311
Epoch 900/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.7230 - val_loss: 6.7967
Epoch 901/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.5495 - val_loss: 6.7376
Epoch 902/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.8495 - val_loss: 7.4873
Epoch 903/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.7096 - val_loss: 6.7642
Epoch 904/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.7310 - val_loss: 6.7619
Epoch 905/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 91us/step - loss: 5.6001 - val_loss: 7.2896
Epoch 973/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.5447 - val_loss: 7.4415
Epoch 974/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.6993 - val_loss: 7.2082
Epoch 975/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.7216 - val_loss: 6.9210
Epoch 976/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.6761 - val_loss: 6.8493
Epoch 977/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.5207 - val_loss: 6.8754
Epoch 978/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.5691 - val_loss: 7.6228
Epoch 979/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.9078 - val_loss: 6.8670
Epoch 980/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.5105 - val_loss: 7.1545
Epoch 981/1000
1052/1052 [==========================

[array([[ 0.09842891, -5.9014864 , -3.633644  , -0.13739903,  2.5892413 ],
        [-1.7494925 , -0.84082913, -1.1324776 , -3.5437012 , -0.96982396],
        [-0.15670197, -1.8609674 ,  2.2376769 , -4.8273067 ,  1.7579633 ],
        [ 0.65256923,  0.48146918,  0.12185725,  0.7327178 , -0.14215545],
        [-1.1810416 , -0.94193864, -3.087126  ,  1.1811019 ,  0.18019107]],
       dtype=float32),
 array([ 4.898565  , -0.7032437 ,  0.58674103, -1.2950473 ,  2.437206  ],
       dtype=float32),
 array([[ -0.9338975 ,   6.3288255 ,   0.34630173,   0.14828701,
           0.33993703,   0.39467466,   1.9059079 ,  -1.3629316 ,
          18.742342  ,   0.66618794],
        [  1.110268  ,   1.4438086 ,   5.7119894 ,  -6.303383  ,
           5.876701  ,   8.677212  ,  -2.588627  ,   0.05822279,
          -0.9374768 ,   2.3301573 ],
        [  1.2054665 ,   2.0089025 ,   0.23766361, -16.303696  ,
           0.24573158,   0.25445515,  -1.0777525 ,   2.7943509 ,
           1.1265295 ,   0.20810513],


In [29]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_sigmoid_4th.h5')

In [30]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_tanh(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 225us/step - l

1052/1052 [==============================] - 0s 91us/step - loss: 47.3711 - val_loss: 43.9291
Epoch 67/1000
1052/1052 [==============================] - 0s 90us/step - loss: 47.3689 - val_loss: 43.9140
Epoch 68/1000
1052/1052 [==============================] - 0s 88us/step - loss: 47.3717 - val_loss: 43.9311
Epoch 69/1000
1052/1052 [==============================] - 0s 88us/step - loss: 47.3672 - val_loss: 43.9194
Epoch 70/1000
1052/1052 [==============================] - 0s 92us/step - loss: 47.3752 - val_loss: 43.9155
Epoch 71/1000
1052/1052 [==============================] - 0s 94us/step - loss: 47.3687 - val_loss: 43.9201
Epoch 72/1000
1052/1052 [==============================] - 0s 91us/step - loss: 47.3675 - val_loss: 43.9132
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 47.3747 - val_loss: 43.9246
Epoch 74/1000
1052/1052 [==============================] - 0s 90us/step - loss: 47.3670 - val_loss: 43.9075
Epoch 75/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 91us/step - loss: 14.6152 - val_loss: 18.4518
Epoch 142/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.6720 - val_loss: 18.2697
Epoch 143/1000
1052/1052 [==============================] - 0s 94us/step - loss: 14.6270 - val_loss: 18.6040
Epoch 144/1000
1052/1052 [==============================] - 0s 93us/step - loss: 14.6644 - val_loss: 18.5033
Epoch 145/1000
1052/1052 [==============================] - 0s 86us/step - loss: 14.6084 - val_loss: 18.1928
Epoch 146/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.6753 - val_loss: 18.2094
Epoch 147/1000
1052/1052 [==============================] - 0s 91us/step - loss: 14.6060 - val_loss: 18.4859
Epoch 148/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.6192 - val_loss: 18.2668
Epoch 149/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.6027 - val_loss: 18.1877
Epoch 150/1000
1052/1052 [========

1052/1052 [==============================] - 0s 100us/step - loss: 14.5559 - val_loss: 18.4799
Epoch 217/1000
1052/1052 [==============================] - 0s 95us/step - loss: 14.6203 - val_loss: 18.1685
Epoch 218/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.6512 - val_loss: 18.3578
Epoch 219/1000
1052/1052 [==============================] - 0s 86us/step - loss: 14.6188 - val_loss: 18.1438
Epoch 220/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.5448 - val_loss: 18.4844
Epoch 221/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.5411 - val_loss: 18.5450
Epoch 222/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.7012 - val_loss: 18.4345
Epoch 223/1000
1052/1052 [==============================] - 0s 93us/step - loss: 14.5587 - val_loss: 18.4325
Epoch 224/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.6210 - val_loss: 18.3532
Epoch 225/1000
1052/1052 [=======

1052/1052 [==============================] - 0s 93us/step - loss: 14.5402 - val_loss: 18.5393
Epoch 292/1000
1052/1052 [==============================] - 0s 83us/step - loss: 14.5934 - val_loss: 18.4130
Epoch 293/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.4542 - val_loss: 18.2923
Epoch 294/1000
1052/1052 [==============================] - 0s 94us/step - loss: 14.6337 - val_loss: 18.1090
Epoch 295/1000
1052/1052 [==============================] - 0s 96us/step - loss: 14.5806 - val_loss: 18.2006
Epoch 296/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.5242 - val_loss: 18.2265
Epoch 297/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.4487 - val_loss: 18.4007
Epoch 298/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.5429 - val_loss: 18.2429
Epoch 299/1000
1052/1052 [==============================] - 0s 94us/step - loss: 14.5970 - val_loss: 18.2467
Epoch 300/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 88us/step - loss: 14.2134 - val_loss: 17.9295
Epoch 742/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.2563 - val_loss: 17.9089
Epoch 743/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.5724 - val_loss: 18.1773
Epoch 744/1000
1052/1052 [==============================] - 0s 91us/step - loss: 14.3243 - val_loss: 17.9344
Epoch 745/1000
1052/1052 [==============================] - 0s 86us/step - loss: 14.1385 - val_loss: 18.0763
Epoch 746/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.1126 - val_loss: 17.8860
Epoch 747/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.0654 - val_loss: 17.9250
Epoch 748/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.2189 - val_loss: 18.0263
Epoch 749/1000
1052/1052 [==============================] - 0s 91us/step - loss: 14.2683 - val_loss: 17.9445
Epoch 750/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 14.0522 - val_loss: 18.1839
Epoch 817/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.0627 - val_loss: 17.7587
Epoch 818/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.0846 - val_loss: 17.6462
Epoch 819/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.9520 - val_loss: 17.8040
Epoch 820/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.9228 - val_loss: 18.1169
Epoch 821/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.0912 - val_loss: 18.1274
Epoch 822/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.9115 - val_loss: 17.7277
Epoch 823/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.9853 - val_loss: 18.0188
Epoch 824/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.9795 - val_loss: 17.8223
Epoch 825/1000
1052/1052 [========

1052/1052 [==============================] - 0s 93us/step - loss: 13.9607 - val_loss: 17.6913
Epoch 892/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.0513 - val_loss: 18.1075
Epoch 893/1000
1052/1052 [==============================] - ETA: 0s - loss: 13.21 - 0s 84us/step - loss: 13.9066 - val_loss: 17.6713
Epoch 894/1000
1052/1052 [==============================] - 0s 91us/step - loss: 14.2318 - val_loss: 17.7872
Epoch 895/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.0645 - val_loss: 17.8295
Epoch 896/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.0102 - val_loss: 17.7555
Epoch 897/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.9002 - val_loss: 17.6605
Epoch 898/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.0552 - val_loss: 17.8348
Epoch 899/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.8586 - val_loss: 17.6126
Epoch 900/

1052/1052 [==============================] - 0s 91us/step - loss: 13.8348 - val_loss: 17.6061
Epoch 967/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.8209 - val_loss: 17.6672
Epoch 968/1000
1052/1052 [==============================] - 0s 91us/step - loss: 13.9314 - val_loss: 17.5682
Epoch 969/1000
1052/1052 [==============================] - 0s 92us/step - loss: 13.8536 - val_loss: 17.6277
Epoch 970/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.9689 - val_loss: 17.6030
Epoch 971/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.8419 - val_loss: 17.7268
Epoch 972/1000
1052/1052 [==============================] - 0s 94us/step - loss: 14.0495 - val_loss: 17.7716
Epoch 973/1000
1052/1052 [==============================] - 0s 92us/step - loss: 14.0606 - val_loss: 17.6107
Epoch 974/1000
1052/1052 [==============================] - 0s 91us/step - loss: 13.9713 - val_loss: 18.0651
Epoch 975/1000
1052/1052 [========

[array([[-0.2129882 , -1.3916765 , -2.498589  , -4.306378  , -0.28662115],
        [-0.25465715, -2.1115909 , -0.76168764, -0.14669733, -1.6031045 ],
        [ 1.2049081 ,  0.11068133, -3.6935475 , -0.05172126, -3.457383  ],
        [-0.52491426,  0.09669611, -0.11740874,  0.5195677 ,  0.76185924],
        [-1.0396574 , -0.33236754, -0.8500821 , -0.23586908,  0.74254096]],
       dtype=float32),
 array([ 0.599656 ,  1.8336166,  0.454187 , -0.6513071, -0.8254293],
       dtype=float32),
 array([[ 17.965073  ,  -1.4654329 ,   0.4134388 ,   1.4050494 ,
           2.1075644 ,   0.3005464 ,  -1.0886469 ,  15.2600765 ,
           1.3145143 ,   3.3079073 ],
        [  6.8659935 ,  -1.0694307 ,  -1.5926831 ,   9.92844   ,
           0.7976873 ,  -0.41765836,  -0.7821657 ,   6.295135  ,
           0.9926483 ,  -1.0106857 ],
        [  7.802627  , -12.214326  ,  -4.57506   ,   1.6078452 ,
          13.531264  ,  -0.20133321,  -8.968804  ,   5.4872236 ,
          11.441814  ,  10.95442   ],
     

In [31]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_tanh_4th.h5')

In [32]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_linear(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 684us/step - loss:

274/274 [==============================] - 0s 102us/step - loss: 6.8844 - val_loss: 9.7785
Epoch 69/1000
274/274 [==============================] - 0s 98us/step - loss: 6.8792 - val_loss: 9.7659
Epoch 70/1000
274/274 [==============================] - 0s 102us/step - loss: 6.7891 - val_loss: 10.0221
Epoch 71/1000
274/274 [==============================] - 0s 95us/step - loss: 6.9409 - val_loss: 9.7190
Epoch 72/1000
274/274 [==============================] - 0s 97us/step - loss: 6.8094 - val_loss: 9.8870
Epoch 73/1000
274/274 [==============================] - 0s 106us/step - loss: 7.0282 - val_loss: 9.9163
Epoch 74/1000
274/274 [==============================] - 0s 102us/step - loss: 6.9788 - val_loss: 9.6680
Epoch 75/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8553 - val_loss: 9.5919
Epoch 76/1000
274/274 [==============================] - 0s 98us/step - loss: 7.0528 - val_loss: 9.6825
Epoch 77/1000
274/274 [==============================] - 0s 101us/step -

Epoch 146/1000
274/274 [==============================] - 0s 99us/step - loss: 6.6007 - val_loss: 9.4913
Epoch 147/1000
274/274 [==============================] - 0s 91us/step - loss: 6.7114 - val_loss: 9.5248
Epoch 148/1000
274/274 [==============================] - 0s 87us/step - loss: 6.6659 - val_loss: 9.3778
Epoch 149/1000
274/274 [==============================] - 0s 91us/step - loss: 6.6050 - val_loss: 9.6321
Epoch 150/1000
274/274 [==============================] - 0s 104us/step - loss: 6.5404 - val_loss: 9.2799
Epoch 151/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5958 - val_loss: 9.5428
Epoch 152/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6216 - val_loss: 9.2847
Epoch 153/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5767 - val_loss: 9.5678
Epoch 154/1000
274/274 [==============================] - 0s 104us/step - loss: 6.9115 - val_loss: 9.4598
Epoch 155/1000
274/274 [==========================

274/274 [==============================] - 0s 109us/step - loss: 6.5600 - val_loss: 9.3782
Epoch 224/1000
274/274 [==============================] - 0s 105us/step - loss: 6.4565 - val_loss: 9.1962
Epoch 225/1000
274/274 [==============================] - 0s 101us/step - loss: 6.5015 - val_loss: 9.3819
Epoch 226/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5610 - val_loss: 9.4138
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6556 - val_loss: 9.5747
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 6.7379 - val_loss: 9.4089
Epoch 229/1000
274/274 [==============================] - 0s 106us/step - loss: 6.8377 - val_loss: 9.4459
Epoch 230/1000
274/274 [==============================] - 0s 106us/step - loss: 6.3987 - val_loss: 9.5345
Epoch 231/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4428 - val_loss: 9.3245
Epoch 232/1000
274/274 [==============================] - 0s 98

274/274 [==============================] - 0s 106us/step - loss: 6.1907 - val_loss: 8.9672
Epoch 301/1000
274/274 [==============================] - 0s 106us/step - loss: 6.2234 - val_loss: 9.0985
Epoch 302/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3004 - val_loss: 9.0596
Epoch 303/1000
274/274 [==============================] - 0s 101us/step - loss: 6.2166 - val_loss: 9.0944
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3150 - val_loss: 9.0598
Epoch 305/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2047 - val_loss: 9.2104
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 6.1886 - val_loss: 8.9974
Epoch 307/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2177 - val_loss: 9.0859
Epoch 308/1000
274/274 [==============================] - 0s 102us/step - loss: 6.1354 - val_loss: 9.0773
Epoch 309/1000
274/274 [==============================] - 0s 98us

...

274/274 [==============================] - 0s 105us/step - loss: 5.6333 - val_loss: 8.5824
Epoch 763/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6321 - val_loss: 8.4042
Epoch 764/1000
274/274 [==============================] - 0s 93us/step - loss: 5.8029 - val_loss: 8.5447
Epoch 765/1000
274/274 [==============================] - 0s 103us/step - loss: 5.7227 - val_loss: 8.4707
Epoch 766/1000
274/274 [==============================] - 0s 97us/step - loss: 5.6746 - val_loss: 8.3822
Epoch 767/1000
274/274 [==============================] - 0s 103us/step - loss: 5.6878 - val_loss: 8.6291
Epoch 768/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7413 - val_loss: 8.4009
Epoch 769/1000
274/274 [==============================] - 0s 100us/step - loss: 5.5980 - val_loss: 8.5405
Epoch 770/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5809 - val_loss: 8.5866
Epoch 771/1000
274/274 [==============================] - 0s 98u

274/274 [==============================] - 0s 95us/step - loss: 5.6905 - val_loss: 8.4437
Epoch 840/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7098 - val_loss: 8.5157
Epoch 841/1000
274/274 [==============================] - 0s 106us/step - loss: 5.6237 - val_loss: 8.5288
Epoch 842/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2042 - val_loss: 8.6430
Epoch 843/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6573 - val_loss: 8.5377
Epoch 844/1000
274/274 [==============================] - 0s 101us/step - loss: 5.5958 - val_loss: 8.5906
Epoch 845/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6019 - val_loss: 8.6228
Epoch 846/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6049 - val_loss: 8.4807
Epoch 847/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6009 - val_loss: 8.5217
Epoch 848/1000
274/274 [==============================] - 0s 95us/

274/274 [==============================] - 0s 113us/step - loss: 5.6135 - val_loss: 8.6146
Epoch 917/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5576 - val_loss: 8.4491
Epoch 918/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6180 - val_loss: 8.7432
Epoch 919/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5470 - val_loss: 8.4470
Epoch 920/1000
274/274 [==============================] - 0s 100us/step - loss: 5.6248 - val_loss: 8.3671
Epoch 921/1000
274/274 [==============================] - 0s 99us/step - loss: 5.5806 - val_loss: 8.3640
Epoch 922/1000
274/274 [==============================] - 0s 106us/step - loss: 5.5631 - val_loss: 8.3697
Epoch 923/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5840 - val_loss: 8.6445
Epoch 924/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5386 - val_loss: 8.4535
Epoch 925/1000
274/274 [==============================] - 0s 102u

274/274 [==============================] - 0s 102us/step - loss: 6.0525 - val_loss: 8.5214
Epoch 994/1000
274/274 [==============================] - 0s 106us/step - loss: 6.1849 - val_loss: 8.9953
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5521 - val_loss: 8.7284
Epoch 996/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0522 - val_loss: 8.9772
Epoch 997/1000
274/274 [==============================] - 0s 105us/step - loss: 5.6153 - val_loss: 8.4773
Epoch 998/1000
274/274 [==============================] - 0s 106us/step - loss: 5.5334 - val_loss: 9.0170
Epoch 999/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6589 - val_loss: 8.4599
Epoch 1000/1000
274/274 [==============================] - 0s 100us/step - loss: 5.5623 - val_loss: 8.5517
7.207257876961918


[array([[ 1.3227559 ,  0.6106956 , -0.8675589 , -3.5244222 , -1.1287844 ],
        [-5.041491  , -2.3730898 ,  0.63648856,  4.074997  , -0.42533156],
        [-1.8531946 ,  1.14759   ,  0.22032951, -2.424565  ,  0.40579486],
        [ 2.3968732 ,  2.6486132 ,  0.53338337, -2.9351628 ,  2.2392395 ],
        [-1.6769222 , -1.1783738 ,  1.1914943 , -0.44460407, -0.42974356],
        [-1.2211828 , -0.99476206, -1.2144927 ,  1.0303143 , -1.4027028 ],
        [ 2.7640762 ,  0.82176155, -1.4117393 ,  2.5781817 , -0.21668957]],
       dtype=float32),
 array([ 1.7787924 , -0.74598217, -2.680197  , -1.210682  ,  2.4426036 ],
       dtype=float32),
 array([[-0.38798124, -0.33427227, -0.10083748,  0.40901184, -0.44767374,
         -0.59402424,  0.12195282,  0.149138  ,  0.5278687 ,  0.06234827],
        [-0.1023507 ,  0.41467944,  0.6408592 , -0.0948898 ,  0.56022304,
          0.43878043, -0.15697199, -0.52051103,  0.14196037,  0.04117951],
        [-0.13972719,  0.19807121,  0.3224786 ,  0.42278

In [33]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_linear_4th.h5')

In [34]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_relu(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 707us/step - loss:

Epoch 69/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1683 - val_loss: 7.2298
Epoch 70/1000
274/274 [==============================] - 0s 109us/step - loss: 7.2456 - val_loss: 7.0825
Epoch 71/1000
274/274 [==============================] - 0s 106us/step - loss: 7.2901 - val_loss: 7.4210
Epoch 72/1000
274/274 [==============================] - 0s 98us/step - loss: 6.9856 - val_loss: 7.2983
Epoch 73/1000
274/274 [==============================] - 0s 98us/step - loss: 7.3164 - val_loss: 7.3068
Epoch 74/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1279 - val_loss: 6.9523
Epoch 75/1000
274/274 [==============================] - 0s 98us/step - loss: 7.2750 - val_loss: 6.9835
Epoch 76/1000
274/274 [==============================] - 0s 109us/step - loss: 7.2021 - val_loss: 7.3953
Epoch 77/1000
274/274 [==============================] - 0s 101us/step - loss: 8.0349 - val_loss: 7.3187
Epoch 78/1000
274/274 [==============================] - 0

274/274 [==============================] - 0s 98us/step - loss: 6.8571 - val_loss: 7.0344
Epoch 147/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6038 - val_loss: 7.0058
Epoch 148/1000
274/274 [==============================] - 0s 91us/step - loss: 6.4912 - val_loss: 6.7746
Epoch 149/1000
274/274 [==============================] - 0s 106us/step - loss: 6.5768 - val_loss: 7.0195
Epoch 150/1000
274/274 [==============================] - 0s 116us/step - loss: 6.4723 - val_loss: 7.1312
Epoch 151/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5118 - val_loss: 6.8704
Epoch 152/1000
274/274 [==============================] - 0s 106us/step - loss: 6.6385 - val_loss: 6.8876
Epoch 153/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6589 - val_loss: 7.2477
Epoch 154/1000
274/274 [==============================] - 0s 109us/step - loss: 6.6301 - val_loss: 7.1422
Epoch 155/1000
274/274 [==============================] - 0s 106

274/274 [==============================] - 0s 102us/step - loss: 6.4729 - val_loss: 7.1665
Epoch 224/1000
274/274 [==============================] - 0s 106us/step - loss: 6.3860 - val_loss: 7.0623
Epoch 225/1000
274/274 [==============================] - 0s 101us/step - loss: 6.3582 - val_loss: 7.1402
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3082 - val_loss: 7.0736
Epoch 227/1000
274/274 [==============================] - 0s 109us/step - loss: 6.3099 - val_loss: 6.7786
Epoch 228/1000
274/274 [==============================] - 0s 106us/step - loss: 6.3860 - val_loss: 7.3330
Epoch 229/1000
274/274 [==============================] - 0s 106us/step - loss: 6.3882 - val_loss: 7.0319
Epoch 230/1000
274/274 [==============================] - 0s 106us/step - loss: 6.4742 - val_loss: 6.7373
Epoch 231/1000
274/274 [==============================] - 0s 109us/step - loss: 6.2726 - val_loss: 7.1913
Epoch 232/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 113us/step - loss: 6.4246 - val_loss: 7.8721
Epoch 301/1000
274/274 [==============================] - 0s 106us/step - loss: 6.5701 - val_loss: 7.0586
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0744 - val_loss: 7.1040
Epoch 303/1000
274/274 [==============================] - 0s 106us/step - loss: 6.2884 - val_loss: 7.4393
Epoch 304/1000
274/274 [==============================] - 0s 111us/step - loss: 6.0583 - val_loss: 7.0468
Epoch 305/1000
274/274 [==============================] - 0s 106us/step - loss: 6.1643 - val_loss: 7.1143
Epoch 306/1000
274/274 [==============================] - 0s 112us/step - loss: 6.1445 - val_loss: 7.2287
Epoch 307/1000
274/274 [==============================] - 0s 100us/step - loss: 6.0421 - val_loss: 6.9613
Epoch 308/1000
274/274 [==============================] - 0s 109us/step - loss: 6.0253 - val_loss: 7.0061
Epoch 309/1000
274/274 [==============================] - 0s 

...

274/274 [==============================] - 0s 98us/step - loss: 5.9282 - val_loss: 7.9652
Epoch 763/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5994 - val_loss: 8.1570
Epoch 764/1000
274/274 [==============================] - 0s 106us/step - loss: 6.5801 - val_loss: 8.9797
Epoch 765/1000
274/274 [==============================] - 0s 100us/step - loss: 6.4459 - val_loss: 8.5097
Epoch 766/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0981 - val_loss: 7.6356
Epoch 767/1000
274/274 [==============================] - 0s 105us/step - loss: 5.7959 - val_loss: 7.0928
Epoch 768/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7059 - val_loss: 7.5299
Epoch 769/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7178 - val_loss: 7.6705
Epoch 770/1000
274/274 [==============================] - ETA: 0s - loss: 5.855 - 0s 106us/step - loss: 5.7615 - val_loss: 7.1977
Epoch 771/1000
274/274 [===============

274/274 [==============================] - 0s 91us/step - loss: 5.6823 - val_loss: 7.3143
Epoch 840/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6182 - val_loss: 7.5809
Epoch 841/1000
274/274 [==============================] - 0s 106us/step - loss: 5.7165 - val_loss: 7.4624
Epoch 842/1000
274/274 [==============================] - 0s 91us/step - loss: 5.8640 - val_loss: 7.2658
Epoch 843/1000
274/274 [==============================] - 0s 106us/step - loss: 5.6548 - val_loss: 7.2102
Epoch 844/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0101 - val_loss: 7.7746
Epoch 845/1000
274/274 [==============================] - 0s 113us/step - loss: 5.8248 - val_loss: 7.8914
Epoch 846/1000
274/274 [==============================] - 0s 104us/step - loss: 6.1002 - val_loss: 8.4682
Epoch 847/1000
274/274 [==============================] - 0s 109us/step - loss: 6.2346 - val_loss: 9.3638
Epoch 848/1000
274/274 [==============================] - 0s 106

Epoch 917/1000
274/274 [==============================] - 0s 99us/step - loss: 5.7809 - val_loss: 7.7305
Epoch 918/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8527 - val_loss: 7.7282
Epoch 919/1000
274/274 [==============================] - 0s 102us/step - loss: 6.1183 - val_loss: 7.6050
Epoch 920/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6579 - val_loss: 7.4417
Epoch 921/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6278 - val_loss: 7.3350
Epoch 922/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9560 - val_loss: 7.8867
Epoch 923/1000
274/274 [==============================] - 0s 99us/step - loss: 5.6084 - val_loss: 8.4024
Epoch 924/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0070 - val_loss: 8.0480
Epoch 925/1000
274/274 [==============================] - 0s 101us/step - loss: 5.8392 - val_loss: 7.4843
Epoch 926/1000
274/274 [=============================

274/274 [==============================] - 0s 98us/step - loss: 5.7270 - val_loss: 7.7912
Epoch 995/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6321 - val_loss: 7.7907
Epoch 996/1000
274/274 [==============================] - 0s 91us/step - loss: 5.5250 - val_loss: 7.8104
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6012 - val_loss: 8.5363
Epoch 998/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7247 - val_loss: 7.2651
Epoch 999/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6051 - val_loss: 7.0903
Epoch 1000/1000
274/274 [==============================] - 0s 109us/step - loss: 5.6366 - val_loss: 7.7862
6.990270040803036


[array([[-0.40470234, -0.34036833, -0.21301723, -2.316638  ,  0.44856235],
        [-0.537564  ,  0.83137596, -1.4813678 ,  1.2403859 ,  1.3589271 ],
        [ 0.09280413, -1.688394  , -0.05514244, -0.45548353, -0.32898977],
        [-0.14545976, -0.9405773 ,  0.7701559 , -0.46130422, -0.88907075],
        [ 0.11598106, -0.05062935, -0.8791152 , -0.10582069,  0.34689492],
        [ 0.4911989 ,  0.40812534, -0.93944454, -0.01865491,  0.9012109 ],
        [ 0.28417295,  1.4655538 , -1.1644257 ,  0.46175352, -1.2777091 ]],
       dtype=float32),
 array([ 2.0805876 , -0.09349127,  0.7071946 , -2.014894  ,  1.0451025 ],
       dtype=float32),
 array([[ 0.7071228 ,  1.236339  ,  0.51266813,  0.7351515 ,  0.5657641 ,
         -0.7828475 , -0.6345413 , -0.2411179 ,  0.61309546, -0.3517006 ],
        [ 1.2454439 ,  0.96782553,  1.2109356 ,  0.7396092 ,  0.13301918,
         -0.65142345, -0.3792337 , -0.8569448 ,  0.5029983 ,  0.8623957 ],
        [ 0.9147484 ,  0.63412374, -0.0052255 , -0.07606

In [35]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_relu_4th.h5')

In [36]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_sigmoid(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 868us/step - loss:

Epoch 68/1000
274/274 [==============================] - 0s 87us/step - loss: 61.2761 - val_loss: 61.6543
Epoch 69/1000
274/274 [==============================] - 0s 102us/step - loss: 61.2753 - val_loss: 61.4300
Epoch 70/1000
274/274 [==============================] - 0s 102us/step - loss: 61.2311 - val_loss: 61.4460
Epoch 71/1000
274/274 [==============================] - 0s 102us/step - loss: 61.2604 - val_loss: 61.5225
Epoch 72/1000
274/274 [==============================] - 0s 108us/step - loss: 61.2470 - val_loss: 61.4423
Epoch 73/1000
274/274 [==============================] - 0s 113us/step - loss: 61.2583 - val_loss: 61.3479
Epoch 74/1000
274/274 [==============================] - 0s 120us/step - loss: 61.2404 - val_loss: 61.4454
Epoch 75/1000
274/274 [==============================] - 0s 117us/step - loss: 61.2320 - val_loss: 61.4560
Epoch 76/1000
274/274 [==============================] - 0s 106us/step - loss: 61.2743 - val_loss: 61.5758
Epoch 77/1000
274/274 [===============

274/274 [==============================] - 0s 102us/step - loss: 22.2782 - val_loss: 22.3461
Epoch 145/1000
274/274 [==============================] - 0s 106us/step - loss: 20.9328 - val_loss: 21.1962
Epoch 146/1000
274/274 [==============================] - 0s 102us/step - loss: 19.6731 - val_loss: 20.0090
Epoch 147/1000
274/274 [==============================] - 0s 102us/step - loss: 18.7517 - val_loss: 19.0944
Epoch 148/1000
274/274 [==============================] - 0s 95us/step - loss: 18.0109 - val_loss: 18.3440
Epoch 149/1000
274/274 [==============================] - 0s 91us/step - loss: 17.2605 - val_loss: 17.7648
Epoch 150/1000
274/274 [==============================] - 0s 98us/step - loss: 16.6277 - val_loss: 17.5008
Epoch 151/1000
274/274 [==============================] - 0s 91us/step - loss: 16.1498 - val_loss: 17.1556
Epoch 152/1000
274/274 [==============================] - 0s 102us/step - loss: 15.5652 - val_loss: 16.7529
Epoch 153/1000
274/274 [=======================

Epoch 221/1000
274/274 [==============================] - 0s 99us/step - loss: 5.7446 - val_loss: 9.6955
Epoch 222/1000
274/274 [==============================] - 0s 87us/step - loss: 5.6852 - val_loss: 9.5850
Epoch 223/1000
274/274 [==============================] - 0s 94us/step - loss: 5.7603 - val_loss: 9.4095
Epoch 224/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7416 - val_loss: 9.5279
Epoch 225/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7487 - val_loss: 9.6823
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7042 - val_loss: 9.6631
Epoch 227/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7216 - val_loss: 9.7982
Epoch 228/1000
274/274 [==============================] - 0s 92us/step - loss: 5.6326 - val_loss: 9.5747
Epoch 229/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6302 - val_loss: 9.4487
Epoch 230/1000
274/274 [==============================]

274/274 [==============================] - 0s 97us/step - loss: 5.2558 - val_loss: 9.2916
Epoch 299/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1840 - val_loss: 9.2943
Epoch 300/1000
274/274 [==============================] - 0s 96us/step - loss: 5.2179 - val_loss: 9.2747
Epoch 301/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1706 - val_loss: 9.3169
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1560 - val_loss: 9.1921
Epoch 303/1000
274/274 [==============================] - 0s 116us/step - loss: 5.1830 - val_loss: 9.1532
Epoch 304/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3406 - val_loss: 9.3444
Epoch 305/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2297 - val_loss: 9.3014
Epoch 306/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1667 - val_loss: 9.1487
Epoch 307/1000
274/274 [==============================] - 0s 98us/s

...

274/274 [==============================] - 0s 102us/step - loss: 4.2521 - val_loss: 8.9274
Epoch 762/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2563 - val_loss: 8.9177
Epoch 763/1000
274/274 [==============================] - 0s 106us/step - loss: 4.2537 - val_loss: 8.9012
Epoch 764/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2550 - val_loss: 8.8706
Epoch 765/1000
274/274 [==============================] - 0s 101us/step - loss: 4.3072 - val_loss: 8.8512
Epoch 766/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2687 - val_loss: 8.7916
Epoch 767/1000
274/274 [==============================] - 0s 95us/step - loss: 4.2347 - val_loss: 8.9297
Epoch 768/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2536 - val_loss: 8.9665
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2534 - val_loss: 8.9675
Epoch 770/1000
274/274 [==============================] - 0s 98u

274/274 [==============================] - 0s 99us/step - loss: 4.2123 - val_loss: 9.0263
Epoch 839/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1618 - val_loss: 9.0074
Epoch 840/1000
274/274 [==============================] - 0s 101us/step - loss: 4.1966 - val_loss: 9.0488
Epoch 841/1000
274/274 [==============================] - 0s 105us/step - loss: 4.2111 - val_loss: 9.0043
Epoch 842/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1666 - val_loss: 8.9503
Epoch 843/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2036 - val_loss: 9.0387
Epoch 844/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1931 - val_loss: 9.0966
Epoch 845/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2023 - val_loss: 8.9087
Epoch 846/1000
274/274 [==============================] - 0s 95us/step - loss: 4.1836 - val_loss: 9.0700
Epoch 847/1000
274/274 [==============================] - 0s 103

274/274 [==============================] - 0s 109us/step - loss: 4.1351 - val_loss: 9.1727
Epoch 916/1000
274/274 [==============================] - 0s 106us/step - loss: 4.1458 - val_loss: 8.9472
Epoch 917/1000
274/274 [==============================] - 0s 100us/step - loss: 4.2551 - val_loss: 9.2095
Epoch 918/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1468 - val_loss: 9.2616
Epoch 919/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1377 - val_loss: 9.0856
Epoch 920/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1349 - val_loss: 9.0782
Epoch 921/1000
274/274 [==============================] - 0s 104us/step - loss: 4.0884 - val_loss: 9.1831
Epoch 922/1000
274/274 [==============================] - 0s 105us/step - loss: 4.1473 - val_loss: 9.1481
Epoch 923/1000
274/274 [==============================] - 0s 95us/step - loss: 4.1221 - val_loss: 9.0983
Epoch 924/1000
274/274 [==============================] - 0s 98u

274/274 [==============================] - 0s 97us/step - loss: 4.1163 - val_loss: 9.2249
Epoch 993/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1230 - val_loss: 9.2155
Epoch 994/1000
274/274 [==============================] - 0s 109us/step - loss: 4.0913 - val_loss: 9.1540
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0935 - val_loss: 9.2332
Epoch 996/1000
274/274 [==============================] - 0s 104us/step - loss: 4.0804 - val_loss: 9.2690
Epoch 997/1000
274/274 [==============================] - 0s 107us/step - loss: 4.0813 - val_loss: 9.2256
Epoch 998/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1153 - val_loss: 9.0943
Epoch 999/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0768 - val_loss: 9.2134
Epoch 1000/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0687 - val_loss: 9.1049
4.816794686398263


[array([[ 1.9426727 , -3.154216  ,  1.2947216 , -0.25928482, -3.776607  ],
        [-1.2941108 , -1.8742912 , -2.4135976 ,  0.8706868 ,  2.3275826 ],
        [-0.7898564 , -0.63225806,  3.9876556 , -0.7132185 , -3.685873  ],
        [-3.4944487 ,  0.06518537,  2.3506112 , -1.3158121 , -0.19374251],
        [-0.09480063, -0.89522725,  1.5488071 ,  0.53425384, -0.9364944 ],
        [ 0.40939808, -0.72733086,  0.10736895,  3.66135   ,  0.4963284 ],
        [-0.59899616, -2.4831371 , -3.8611042 , -1.5163778 ,  0.52634037]],
       dtype=float32),
 array([-0.7506606 ,  0.64207524,  2.5996542 ,  0.41672248, -1.3979558 ],
       dtype=float32),
 array([[ 0.99948496,  2.688472  ,  1.2906566 ,  3.5223038 , -3.1821342 ,
         -2.501884  ,  1.7030418 ,  0.70416105,  2.4237258 ,  1.0781857 ],
        [ 0.83689904,  6.774279  ,  0.427616  , -1.4888628 ,  1.3295146 ,
          0.3013371 , -1.4857911 ,  0.6392784 , -2.1652515 ,  0.78460735],
        [-7.826157  , -0.22110789, -2.2029898 ,  2.55714

In [37]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_sigmoid_4th.h5')

In [38]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_tanh(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 821us/step - loss:

Epoch 68/1000
274/274 [==============================] - 0s 109us/step - loss: 61.0646 - val_loss: 61.0550
Epoch 69/1000
274/274 [==============================] - 0s 109us/step - loss: 61.0680 - val_loss: 61.0739
Epoch 70/1000
274/274 [==============================] - 0s 106us/step - loss: 61.0526 - val_loss: 61.1262
Epoch 71/1000
274/274 [==============================] - 0s 104us/step - loss: 61.0614 - val_loss: 61.0885
Epoch 72/1000
274/274 [==============================] - 0s 103us/step - loss: 61.0620 - val_loss: 61.0463
Epoch 73/1000
274/274 [==============================] - 0s 98us/step - loss: 61.0638 - val_loss: 61.0382
Epoch 74/1000
274/274 [==============================] - 0s 109us/step - loss: 61.0770 - val_loss: 61.0033
Epoch 75/1000
274/274 [==============================] - 0s 105us/step - loss: 61.0570 - val_loss: 60.9768
Epoch 76/1000
274/274 [==============================] - 0s 106us/step - loss: 61.0846 - val_loss: 60.9154
Epoch 77/1000
274/274 [===============

274/274 [==============================] - 0s 102us/step - loss: 8.7929 - val_loss: 11.1905
Epoch 145/1000
274/274 [==============================] - 0s 109us/step - loss: 8.2126 - val_loss: 11.6331
Epoch 146/1000
274/274 [==============================] - 0s 99us/step - loss: 7.7892 - val_loss: 11.1442
Epoch 147/1000
274/274 [==============================] - 0s 102us/step - loss: 7.5104 - val_loss: 11.2881
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 7.4573 - val_loss: 11.0387
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 7.3541 - val_loss: 11.0288
Epoch 150/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1710 - val_loss: 11.0435
Epoch 151/1000
274/274 [==============================] - 0s 101us/step - loss: 7.1035 - val_loss: 10.9901
Epoch 152/1000
274/274 [==============================] - 0s 102us/step - loss: 7.0017 - val_loss: 10.9228
Epoch 153/1000
274/274 [==============================

Epoch 221/1000
274/274 [==============================] - 0s 106us/step - loss: 5.6742 - val_loss: 10.7059
Epoch 222/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8825 - val_loss: 10.4405
Epoch 223/1000
274/274 [==============================] - 0s 95us/step - loss: 6.1004 - val_loss: 10.3137
Epoch 224/1000
274/274 [==============================] - 0s 101us/step - loss: 5.8670 - val_loss: 10.5076
Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8367 - val_loss: 10.4253
Epoch 226/1000
274/274 [==============================] - 0s 138us/step - loss: 5.7724 - val_loss: 10.4563
Epoch 227/1000
274/274 [==============================] - 0s 109us/step - loss: 5.7341 - val_loss: 10.1489
Epoch 228/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6997 - val_loss: 10.2607
Epoch 229/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7247 - val_loss: 10.2212
Epoch 230/1000
274/274 [==================

274/274 [==============================] - 0s 116us/step - loss: 5.2581 - val_loss: 8.2507
Epoch 299/1000
274/274 [==============================] - 0s 120us/step - loss: 5.0886 - val_loss: 8.4315
Epoch 300/1000
274/274 [==============================] - 0s 110us/step - loss: 5.2003 - val_loss: 8.3357
Epoch 301/1000
274/274 [==============================] - 0s 113us/step - loss: 5.1819 - val_loss: 8.4352
Epoch 302/1000
274/274 [==============================] - 0s 116us/step - loss: 5.4509 - val_loss: 8.5213
Epoch 303/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0986 - val_loss: 8.3697
Epoch 304/1000
274/274 [==============================] - 0s 109us/step - loss: 5.1663 - val_loss: 8.4479
Epoch 305/1000
274/274 [==============================] - 0s 109us/step - loss: 5.1817 - val_loss: 8.4381
Epoch 306/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0873 - val_loss: 7.8257
Epoch 307/1000
274/274 [==============================] - 0s 

...

274/274 [==============================] - 0s 102us/step - loss: 4.2255 - val_loss: 6.0421
Epoch 761/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1556 - val_loss: 6.3018
Epoch 762/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2392 - val_loss: 6.2029
Epoch 763/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1573 - val_loss: 6.2993
Epoch 764/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1970 - val_loss: 6.0495
Epoch 765/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2400 - val_loss: 6.1204
Epoch 766/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2345 - val_loss: 6.0784
Epoch 767/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2922 - val_loss: 6.4509
Epoch 768/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2984 - val_loss: 5.8712
Epoch 769/1000
274/274 [==============================] - 0s 102

274/274 [==============================] - 0s 109us/step - loss: 4.1223 - val_loss: 6.8451
Epoch 838/1000
274/274 [==============================] - 0s 109us/step - loss: 4.0460 - val_loss: 6.6800
Epoch 839/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0431 - val_loss: 6.9638
Epoch 840/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0983 - val_loss: 6.7447
Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1838 - val_loss: 7.1137
Epoch 842/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1632 - val_loss: 6.6038
Epoch 843/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1395 - val_loss: 7.0039
Epoch 844/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0554 - val_loss: 6.5791
Epoch 845/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1103 - val_loss: 6.5358
Epoch 846/1000
274/274 [==============================] - 0s 98u

274/274 [==============================] - 0s 95us/step - loss: 3.9185 - val_loss: 7.4363
Epoch 915/1000
274/274 [==============================] - 0s 106us/step - loss: 3.8968 - val_loss: 7.5187
Epoch 916/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9601 - val_loss: 7.2008
Epoch 917/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8598 - val_loss: 7.1457
Epoch 918/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8715 - val_loss: 7.2058
Epoch 919/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9044 - val_loss: 7.2906
Epoch 920/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9027 - val_loss: 7.2434
Epoch 921/1000
274/274 [==============================] - 0s 95us/step - loss: 3.8712 - val_loss: 7.2574
Epoch 922/1000
274/274 [==============================] - 0s 95us/step - loss: 3.8925 - val_loss: 7.1338
Epoch 923/1000
274/274 [==============================] - 0s 95us/s

274/274 [==============================] - 0s 91us/step - loss: 3.8769 - val_loss: 7.2134
Epoch 992/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8627 - val_loss: 7.4632
Epoch 993/1000
274/274 [==============================] - 0s 95us/step - loss: 3.7761 - val_loss: 6.8155
Epoch 994/1000
274/274 [==============================] - 0s 95us/step - loss: 3.8235 - val_loss: 7.3052
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8693 - val_loss: 7.2435
Epoch 996/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0226 - val_loss: 6.8686
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8932 - val_loss: 7.7644
Epoch 998/1000
274/274 [==============================] - 0s 106us/step - loss: 4.0472 - val_loss: 7.2254
Epoch 999/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8667 - val_loss: 6.7731
Epoch 1000/1000
274/274 [==============================] - 0s 98us/

[array([[-2.602393  ,  0.02829075, -0.05768069,  0.18894286,  0.0348461 ],
        [ 0.36711708,  0.16390473,  1.7725432 , -1.6706893 , -0.09129847],
        [-1.2150987 ,  0.6919538 ,  0.4303669 ,  0.04038908, -1.3941375 ],
        [ 0.80949277,  0.41154438, -0.6549829 , -0.14798775, -2.2688634 ],
        [-0.12744196,  0.906149  ,  0.6035765 , -0.08853006,  0.6902179 ],
        [-1.1379122 , -0.36782324,  2.4535136 ,  1.0528342 , -0.46240628],
        [ 0.12341176, -2.4882882 ,  1.2593265 , -2.3518713 ,  0.0658196 ]],
       dtype=float32),
 array([ 0.4970944 ,  0.70481455, -2.4729645 , -0.31301773, -0.16426153],
       dtype=float32),
 array([[-2.7211897 ,  1.1551969 ,  0.83814   ,  0.45088214, -1.261206  ,
          0.57161826,  0.81634265,  2.6362574 , -0.22535393,  3.8579538 ],
        [-0.3434773 ,  0.12222189,  0.80744827,  0.37591806, -1.1984783 ,
         -0.4075547 , -1.0825627 ,  0.36921167, -1.3944631 ,  0.6929098 ],
        [-0.02752915,  0.67814803, -3.3144116 , -1.84156

In [39]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_tanh_4th.h5')

In [40]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_linear(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_18 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 0

143/143 [==============================] - 0s 105us/step - loss: 0.0057 - val_loss: 0.0050
Epoch 70/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0062 - val_loss: 0.0051
Epoch 71/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0052 - val_loss: 0.0039
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0053 - val_loss: 0.0037
Epoch 73/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0052 - val_loss: 0.0043
Epoch 74/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0050 - val_loss: 0.0040
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0049 - val_loss: 0.0042
Epoch 76/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0048 - val_loss: 0.0040
Epoch 77/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0052 - val_loss: 0.0039
Epoch 78/1000
143/143 [==============================] - 0s 112us/ste

143/143 [==============================] - 0s 112us/step - loss: 0.0038 - val_loss: 0.0036
Epoch 147/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0042 - val_loss: 0.0043
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0039 - val_loss: 0.0030
Epoch 149/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0031
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0031
Epoch 151/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0035 - val_loss: 0.0032
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0030
Epoch 153/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0037 - val_loss: 0.0034
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0031
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 109us/step - loss: 0.0028 - val_loss: 0.0038
Epoch 224/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0031 - val_loss: 0.0025
Epoch 225/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0032 - val_loss: 0.0031
Epoch 226/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0032 - val_loss: 0.0030
Epoch 227/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0029 - val_loss: 0.0026
Epoch 228/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0035
Epoch 229/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0033
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0038 - val_loss: 0.0036
Epoch 231/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0036
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0028 - val_loss: 0.0031
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0031
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0026 - val_loss: 0.0027
Epoch 303/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0029
Epoch 304/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0028 - val_loss: 0.0035
Epoch 305/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0029 - val_loss: 0.0023
Epoch 306/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0025 - val_loss: 0.0023
Epoch 307/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0027
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0024 - val_loss: 0.0025
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0027
Epoch 763/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0029
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0040
Epoch 765/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0019 - val_loss: 0.0029
Epoch 766/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0016 - val_loss: 0.0029
Epoch 767/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0036
Epoch 768/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0016 - val_loss: 0.0029
Epoch 769/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0018 - val_loss: 0.0028
Epoch 770/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0016 - val_loss: 0.0029
Epoch 771/1000
143/143 [==============================] - 0s 

Epoch 839/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0014 - val_loss: 0.0031
Epoch 840/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 0.0034
Epoch 841/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0014 - val_loss: 0.0033
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0034
Epoch 843/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0013 - val_loss: 0.0030
Epoch 844/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0032
Epoch 845/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0013 - val_loss: 0.0032
Epoch 846/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0014 - val_loss: 0.0032
Epoch 847/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 0.0031
Epoch 848/1000
143/143 [======================

143/143 [==============================] - 0s 160us/step - loss: 0.0013 - val_loss: 0.0029
Epoch 917/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 0.0033
Epoch 918/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 0.0039
Epoch 919/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0036
Epoch 920/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0018 - val_loss: 0.0037
Epoch 921/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0030
Epoch 922/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0030
Epoch 923/1000
143/143 [==============================] - 0s 153us/step - loss: 0.0012 - val_loss: 0.0038
Epoch 924/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0016 - val_loss: 0.0031
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0031
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0047
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0036
Epoch 997/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0042
Epoch 998/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0028
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0031
Epoch 1000/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0035
0.01423204317688942


[array([[ 0.77011985, -0.13769837, -0.389316  , -0.2538361 , -0.223971  ],
        [ 0.08921811,  0.0023488 ,  1.1988966 , -0.6461715 , -0.05000739],
        [-1.2783828 , -0.0096215 , -0.24032642,  0.6306417 , -0.17771125],
        [-0.53668785,  0.0428123 ,  0.3491278 ,  0.23928878, -0.3634658 ],
        [-0.12259349,  1.1676067 , -1.1196691 ,  0.17181726, -0.22769843],
        [ 0.76557314, -0.3997987 ,  0.46486834, -0.40649703,  0.53394455],
        [-0.4209577 ,  0.1317288 ,  1.5598445 , -0.40733224,  0.35997045],
        [-0.32572052, -0.04426997,  0.35625598,  0.02966321,  0.24931712],
        [-0.29078418,  0.5018781 , -0.47523856,  0.5272559 ,  0.77191764],
        [ 0.7485471 , -0.19148222,  1.2126558 , -0.99541795, -0.2157849 ],
        [-0.18963644,  0.9253079 ,  0.04138198, -0.02924821, -0.43484977],
        [ 0.29253843, -0.7553612 , -1.1454885 ,  1.9062465 ,  0.31471717],
        [ 0.40441692, -0.76354337, -0.47684422, -0.40905067, -0.01620788],
        [ 1.3591845 ,  1.

In [41]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_linear_4th.h5')

In [42]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_relu(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_19 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_20 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 138us/step - loss: 0.0076 - val_loss: 0.0058
Epoch 70/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0067 - val_loss: 0.0071
Epoch 71/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0065 - val_loss: 0.0062
Epoch 72/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0064 - val_loss: 0.0063
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0065 - val_loss: 0.0065
Epoch 74/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0063 - val_loss: 0.0060
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0064 - val_loss: 0.0067
Epoch 76/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0065 - val_loss: 0.0063
Epoch 77/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0069 - val_loss: 0.0058
Epoch 78/1000
143/143 [==============================] - 0s 112us/ste

143/143 [==============================] - 0s 98us/step - loss: 0.0057 - val_loss: 0.0061
Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0056 - val_loss: 0.0058
Epoch 148/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0057 - val_loss: 0.0061
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0055 - val_loss: 0.0066
Epoch 150/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0057 - val_loss: 0.0059
Epoch 151/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0056 - val_loss: 0.0062
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0056 - val_loss: 0.0059
Epoch 153/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0057 - val_loss: 0.0062
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0060 - val_loss: 0.0059
Epoch 155/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 146us/step - loss: 0.0062 - val_loss: 0.0057
Epoch 224/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0054 - val_loss: 0.0065
Epoch 225/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0059 - val_loss: 0.0056
Epoch 226/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0053 - val_loss: 0.0065
Epoch 227/1000
143/143 [==============================] - 0s 244us/step - loss: 0.0054 - val_loss: 0.0056
Epoch 228/1000
143/143 [==============================] - 0s 237us/step - loss: 0.0055 - val_loss: 0.0063
Epoch 229/1000
143/143 [==============================] - 0s 195us/step - loss: 0.0063 - val_loss: 0.0066
Epoch 230/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0051 - val_loss: 0.0054
Epoch 231/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0054 - val_loss: 0.0067
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0050 - val_loss: 0.0058
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0049 - val_loss: 0.0055
Epoch 302/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0050 - val_loss: 0.0054
Epoch 303/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0049 - val_loss: 0.0059
Epoch 304/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0050 - val_loss: 0.0053
Epoch 305/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0051 - val_loss: 0.0055
Epoch 306/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0049 - val_loss: 0.0053
Epoch 307/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0051 - val_loss: 0.0053
Epoch 308/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0049 - val_loss: 0.0052
Epoch 309/1000
143/143 [==============================] - 0s 

...

Epoch 762/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0048 - val_loss: 0.0052
Epoch 763/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0048 - val_loss: 0.0060
Epoch 764/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0047 - val_loss: 0.0053
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0043 - val_loss: 0.0051
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0055
Epoch 767/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0048 - val_loss: 0.0053
Epoch 768/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0061
Epoch 769/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0051
Epoch 770/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0052
Epoch 771/1000
143/143 [======================

143/143 [==============================] - 0s 126us/step - loss: 0.0043 - val_loss: 0.0049
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0044 - val_loss: 0.0053
Epoch 841/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0052 - val_loss: 0.0051
Epoch 842/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0058 - val_loss: 0.0053
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0048 - val_loss: 0.0059
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0043 - val_loss: 0.0049
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0049 - val_loss: 0.0053
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0047 - val_loss: 0.0049
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0042 - val_loss: 0.0058
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0042 - val_loss: 0.0060
Epoch 917/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0051
Epoch 918/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0044 - val_loss: 0.0062
Epoch 919/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0053
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0054
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0054
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0056
Epoch 923/1000
143/143 [==============================] - 0s 166us/step - loss: 0.0047 - val_loss: 0.0052
Epoch 924/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0049 - val_loss: 0.0053
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0043 - val_loss: 0.0053
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0044 - val_loss: 0.0052
Epoch 995/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0045 - val_loss: 0.0056
Epoch 996/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0045 - val_loss: 0.0053
Epoch 997/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0042 - val_loss: 0.0054
Epoch 998/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0043 - val_loss: 0.0055
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0043 - val_loss: 0.0053
Epoch 1000/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0045 - val_loss: 0.0054
0.009161117486655712


[array([[ 0.27699775, -0.11898243, -0.1849051 , -0.35272053, -0.29750234],
        [-0.09878635, -0.1995936 ,  0.11862897, -0.6791794 , -0.45516005],
        [-0.21170785, -0.6452235 ,  0.3015029 , -0.13705756, -0.68083054],
        [ 0.10301439, -0.28390822, -0.7136075 , -0.59993064, -0.32103184],
        [-0.07862579, -0.32691836, -0.16447996, -0.10245146, -0.33501786],
        [ 0.4859878 ,  0.16239613, -0.08858379, -0.7230032 ,  0.01499158],
        [-0.16386111, -0.3686412 , -0.5145171 , -0.3729404 ,  0.04434313],
        [ 0.13794753, -0.24486601, -0.6222135 , -0.1241038 , -0.28423244],
        [-0.1279248 , -0.6881453 , -0.02266933,  0.1662033 , -0.22163618],
        [ 0.4761381 ,  0.17807613,  0.3697165 , -0.35462818,  0.0276014 ],
        [-0.6101771 , -0.53002983, -0.4555932 ,  0.1663567 ,  0.18200853],
        [ 0.11236661, -0.6064761 , -0.5279909 , -0.135238  ,  0.0280204 ],
        [-0.23301362, -0.66832936, -0.14710973, -0.37173527,  0.03468972],
        [ 2.3423698 , -0.

In [43]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_relu_4th.h5')

In [44]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_sigmoid(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_21 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 119us/step - loss: 0.0057 - val_loss: 0.0055
Epoch 70/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0072 - val_loss: 0.0086
Epoch 71/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0069 - val_loss: 0.0054
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0073 - val_loss: 0.0055
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0069 - val_loss: 0.0045
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0079 - val_loss: 0.0052
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0083 - val_loss: 0.0082
Epoch 76/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0079 - val_loss: 0.0044
Epoch 77/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0061 - val_loss: 0.0049
Epoch 78/1000
143/143 [==============================] - 0s 119us/ste

143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0034
Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0035
Epoch 148/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0034
Epoch 149/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0047 - val_loss: 0.0032
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0045 - val_loss: 0.0032
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0049 - val_loss: 0.0049
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0051 - val_loss: 0.0044
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0053 - val_loss: 0.0055
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0047 - val_loss: 0.0032
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0037 - val_loss: 0.0028
Epoch 224/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0025
Epoch 225/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0041 - val_loss: 0.0038
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0025
Epoch 227/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0039 - val_loss: 0.0026
Epoch 228/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0038 - val_loss: 0.0024
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0037 - val_loss: 0.0030
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0038 - val_loss: 0.0025
Epoch 231/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0039 - val_loss: 0.0026
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 153us/step - loss: 0.0035 - val_loss: 0.0022
Epoch 301/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0042 - val_loss: 0.0047
Epoch 302/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0044 - val_loss: 0.0028
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0027
Epoch 304/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0025
Epoch 305/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0046
Epoch 306/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0058 - val_loss: 0.0031
Epoch 307/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0045 - val_loss: 0.0022
Epoch 308/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0036 - val_loss: 0.0021
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 111us/step - loss: 0.0026 - val_loss: 0.0024
Epoch 763/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0035
Epoch 764/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0031 - val_loss: 0.0019
Epoch 765/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0025 - val_loss: 0.0019
Epoch 766/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 767/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0023 - val_loss: 0.0018
Epoch 768/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0024 - val_loss: 0.0017
Epoch 769/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0023
Epoch 770/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0020
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0026 - val_loss: 0.0020
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0026 - val_loss: 0.0019
Epoch 841/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0024 - val_loss: 0.0022
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0019
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0019
Epoch 846/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0019
Epoch 847/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0021
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0019
Epoch 917/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0023
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0020
Epoch 919/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0020
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0020
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0024 - val_loss: 0.0023
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 924/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0021
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0020
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0019
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0019
Epoch 996/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0023 - val_loss: 0.0024
Epoch 997/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0024 - val_loss: 0.0020
Epoch 998/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0024 - val_loss: 0.0021
Epoch 999/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0019
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0021 - val_loss: 0.0026
0.012258177623152733


[array([[-5.13971508e-01, -8.73917520e-01,  7.02551365e-01,
          3.56984437e-02,  1.97673336e-01],
        [-3.68911237e-01, -1.96639985e-01, -3.85284096e-01,
         -8.48502040e-01, -4.27544922e-01],
        [-2.61735290e-01, -1.91894129e-01, -1.49801761e-01,
         -7.85320252e-03,  1.45972162e-01],
        [ 3.97986084e-01, -4.42281395e-01,  1.93476871e-01,
         -7.18864322e-01,  3.80972505e-01],
        [-4.25278217e-01,  9.80827734e-02, -3.69470209e-01,
         -8.91145289e-01, -1.07779056e-01],
        [ 2.13314686e-03, -6.27325356e-01,  9.25354481e-01,
         -5.92765629e-01,  2.18107790e-01],
        [-4.29032296e-02, -4.02274042e-01, -2.92055637e-01,
         -5.63374996e-01, -1.34276533e+00],
        [-6.51183486e-01,  3.39411162e-02, -9.26785320e-02,
          4.97012511e-02,  1.57500759e-01],
        [-5.10955215e-01, -3.23823959e-01,  2.49338686e-01,
         -1.79187089e-01, -2.71118671e-01],
        [-3.14902812e-01, -2.21453775e-02, -7.23139569e-02,
    

In [45]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_sigmoid_4th.h5')

In [46]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_tanh(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_23 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_24 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0262
Epoch 70/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0275
Epoch 71/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0374 - val_loss: 0.0263
Epoch 72/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0372 - val_loss: 0.0276
Epoch 73/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0370 - val_loss: 0.0261
Epoch 74/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0369 - val_loss: 0.0274
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0273
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0280
Epoch 77/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0266
Epoch 78/1000
143/143 [==============================] - 0s 118us/ste

143/143 [==============================] - 0s 123us/step - loss: 0.0369 - val_loss: 0.0273
Epoch 147/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0368 - val_loss: 0.0276
Epoch 148/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0369 - val_loss: 0.0277
Epoch 149/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0369 - val_loss: 0.0275
Epoch 150/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0369 - val_loss: 0.0266
Epoch 151/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0370 - val_loss: 0.0264
Epoch 152/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0369 - val_loss: 0.0279
Epoch 153/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0373 - val_loss: 0.0267
Epoch 154/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0367 - val_loss: 0.0284
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0263
Epoch 224/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0370 - val_loss: 0.0277
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0278
Epoch 226/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0368 - val_loss: 0.0270
Epoch 227/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0369 - val_loss: 0.0264
Epoch 228/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0371 - val_loss: 0.0274
Epoch 229/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0369 - val_loss: 0.0269
Epoch 230/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0371 - val_loss: 0.0279
Epoch 231/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0266
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0370 - val_loss: 0.0283
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0260
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0263
Epoch 304/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0272
Epoch 305/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0275
Epoch 306/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0268
Epoch 307/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0261
Epoch 308/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0371 - val_loss: 0.0260
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 118us/step - loss: 0.0369 - val_loss: 0.0267
Epoch 763/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0268
Epoch 764/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 765/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0274
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0272
Epoch 767/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 768/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 769/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 770/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0367 - val_loss: 0.0268
Epoch 840/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0368 - val_loss: 0.0260
Epoch 841/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0370 - val_loss: 0.0262
Epoch 842/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0369 - val_loss: 0.0270
Epoch 843/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0277
Epoch 844/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0368 - val_loss: 0.0277
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0367 - val_loss: 0.0263
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0259
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 146us/step - loss: 0.0369 - val_loss: 0.0270
Epoch 917/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0368 - val_loss: 0.0272
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 919/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0368 - val_loss: 0.0266
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0269
Epoch 921/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 922/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0368 - val_loss: 0.0273
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 109us/step - loss: 0.0369 - val_loss: 0.0279
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0369 - val_loss: 0.0275
Epoch 995/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0368 - val_loss: 0.0274
Epoch 996/1000
143/143 [==============================] - 0s 127us/step - loss: 0.0368 - val_loss: 0.0271
Epoch 997/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0368 - val_loss: 0.0268
Epoch 998/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0369 - val_loss: 0.0267
Epoch 999/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0368 - val_loss: 0.0272
Epoch 1000/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0369 - val_loss: 0.0277
0.05466809496283531


[array([[-0.7362302 , -0.34700462, -0.21741125,  0.10767622,  0.1187201 ],
        [ 0.15514073, -0.13727006, -0.36625674, -0.7738427 , -0.24979556],
        [-0.16641387, -0.787171  ,  0.07648029, -0.7710066 , -0.3812341 ],
        [-0.27433494,  0.09526598, -0.12003659,  0.4616266 , -0.10050523],
        [-0.2565565 ,  0.12006301, -0.4283556 , -0.6810467 , -0.74825037],
        [-0.53693134, -0.1402363 , -0.5823151 , -0.22366117, -0.7371609 ],
        [-0.02991432,  0.03270359, -0.5839709 , -0.37584418, -0.33775702],
        [-0.7393341 , -0.7265014 ,  0.09733699, -0.1441744 , -0.04916781],
        [ 0.04535153, -0.48583838, -0.7888207 , -0.48617864, -0.34668413],
        [ 0.07686548, -0.6614542 ,  0.02235546, -0.27742395, -0.23424542],
        [-0.38731056, -0.5159101 , -0.15201674, -0.54715705, -0.11502233],
        [-0.6899172 ,  0.15163928, -0.61180955, -0.37086022, -0.07552928],
        [-0.77343833, -0.11249179, -0.34697387,  0.06881394, -0.6452153 ],
        [-0.76510584, -0.

In [47]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_tanh_4th.h5')

In [48]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_linear(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_25 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_26 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 9ms/step - loss: 37.958

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0140 - val_loss: 0.0425
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0142 - val_loss: 0.0395
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0144 - val_loss: 0.0364
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0141 - val_loss: 0.0331
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0133 - val_loss: 0.0299
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0122 - val_loss: 0.0270
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0111 - val_loss: 0.0247
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0104 - val_loss: 0.0229
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0101 - val_loss: 0.0215
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0068
Epoch 152/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0037 - val_loss: 0.0067
Epoch 153/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0037 - val_loss: 0.0066
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0037 - val_loss: 0.0065
Epoch 155/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0036 - val_loss: 0.0064
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0036 - val_loss: 0.0063
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0062
Epoch 158/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0035 - val_loss: 0.0062
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0034 - val_loss: 0.0061
Epoch 160/1000
32/32 [==============================] - 0s 125us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0017 - val_loss: 0.0031
Epoch 231/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0017 - val_loss: 0.0030
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0017 - val_loss: 0.0030
Epoch 233/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0016 - val_loss: 0.0030
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 0.0030
Epoch 235/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0016 - val_loss: 0.0030
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 0.0029
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 0.0029
Epoch 238/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0016 - val_loss: 0.0029
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 309/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0010 - val_loss: 0.0019
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 0.0019
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 0.0019
Epoch 312/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0010 - val_loss: 0.0019
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 0.0019
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 0.0019
Epoch 315/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0010 - val_loss: 0.0019
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 0.0019
Epoch 317/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 0.0018
Epoch 318/1000
32/32 [==============================] - 0s 156us

...

Epoch 759/1000
32/32 [==============================] - 0s 155us/step - loss: 2.1952e-04 - val_loss: 4.8013e-04
Epoch 760/1000
32/32 [==============================] - 0s 125us/step - loss: 2.1891e-04 - val_loss: 4.7898e-04
Epoch 761/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1830e-04 - val_loss: 4.7779e-04
Epoch 762/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1769e-04 - val_loss: 4.7663e-04
Epoch 763/1000
32/32 [==============================] - 0s 125us/step - loss: 2.1708e-04 - val_loss: 4.7548e-04
Epoch 764/1000
32/32 [==============================] - 0s 125us/step - loss: 2.1648e-04 - val_loss: 4.7433e-04
Epoch 765/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1588e-04 - val_loss: 4.7317e-04
Epoch 766/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1527e-04 - val_loss: 4.7203e-04
Epoch 767/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1468e-04 - val_loss: 4.7

Epoch 833/1000
32/32 [==============================] - 0s 187us/step - loss: 1.7923e-04 - val_loss: 4.0195e-04
Epoch 834/1000
32/32 [==============================] - 0s 94us/step - loss: 1.7874e-04 - val_loss: 4.0101e-04
Epoch 835/1000
32/32 [==============================] - 0s 125us/step - loss: 1.7827e-04 - val_loss: 4.0006e-04
Epoch 836/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7779e-04 - val_loss: 3.9911e-04
Epoch 837/1000
32/32 [==============================] - 0s 125us/step - loss: 1.7731e-04 - val_loss: 3.9816e-04
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7684e-04 - val_loss: 3.9722e-04
Epoch 839/1000
32/32 [==============================] - 0s 125us/step - loss: 1.7636e-04 - val_loss: 3.9628e-04
Epoch 840/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7589e-04 - val_loss: 3.9535e-04
Epoch 841/1000
32/32 [==============================] - 0s 125us/step - loss: 1.7542e-04 - val_loss: 3.94

Epoch 907/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4736e-04 - val_loss: 3.3809e-04
Epoch 908/1000
32/32 [==============================] - 0s 125us/step - loss: 1.4698e-04 - val_loss: 3.3731e-04
Epoch 909/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4660e-04 - val_loss: 3.3654e-04
Epoch 910/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4622e-04 - val_loss: 3.3575e-04
Epoch 911/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4584e-04 - val_loss: 3.3498e-04
Epoch 912/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4546e-04 - val_loss: 3.3421e-04
Epoch 913/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4508e-04 - val_loss: 3.3344e-04
Epoch 914/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4471e-04 - val_loss: 3.3266e-04
Epoch 915/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4433e-04 - val_loss: 3.3

Epoch 981/1000
32/32 [==============================] - 0s 125us/step - loss: 1.2185e-04 - val_loss: 2.8536e-04
Epoch 982/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2154e-04 - val_loss: 2.8470e-04
Epoch 983/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2123e-04 - val_loss: 2.8406e-04
Epoch 984/1000
32/32 [==============================] - 0s 125us/step - loss: 1.2093e-04 - val_loss: 2.8341e-04
Epoch 985/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2062e-04 - val_loss: 2.8278e-04
Epoch 986/1000
32/32 [==============================] - 0s 125us/step - loss: 1.2031e-04 - val_loss: 2.8214e-04
Epoch 987/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2001e-04 - val_loss: 2.8151e-04
Epoch 988/1000
32/32 [==============================] - 0s 124us/step - loss: 1.1971e-04 - val_loss: 2.8087e-04
Epoch 989/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1941e-04 - val_loss: 2.8

[array([[-0.03381181, -0.3474659 ,  0.3099932 ,  0.04641125, -0.54604703],
        [ 1.6053275 ,  1.123105  ,  1.3587947 , -0.01824942,  0.04935107],
        [-0.8546664 , -0.38093972, -0.1344669 , -0.0933515 ,  0.06211767]],
       dtype=float32),
 array([-0.4976979 ,  0.20110993, -0.5414311 , -0.6343815 , -0.3522789 ],
       dtype=float32),
 array([[ 0.07417494, -0.41589704,  0.55593395, -0.00704514,  0.51312464,
          0.41571623,  0.3063983 ,  0.19992325,  0.16977036,  0.28271267],
        [ 0.45295957,  0.16003783, -0.16648567, -0.24375196, -0.51488686,
         -0.53548414,  0.5703676 ,  0.5804981 , -0.21556725,  0.21207303],
        [ 0.34049514,  0.16236426, -0.35982284, -0.15736818, -0.3317503 ,
         -0.2867387 ,  0.31016004, -0.3557103 ,  0.10073322, -0.5229236 ],
        [-0.2679118 ,  0.44353116, -0.37480614,  0.3137848 ,  0.3577983 ,
         -0.4050256 , -0.5077242 , -0.35024166,  0.4548997 ,  0.353473  ],
        [-0.33070466,  0.23976596,  0.14400178,  0.3123424

In [49]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_linear_4th.h5')

In [50]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_relu(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_27 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_41 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_28 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 9ms/step - loss: 36.184

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0302 - val_loss: 0.0211
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0270 - val_loss: 0.0192
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0243 - val_loss: 0.0181
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0224 - val_loss: 0.0178
Epoch 75/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0213 - val_loss: 0.0182
Epoch 76/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0208 - val_loss: 0.0188
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0205 - val_loss: 0.0192
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0201 - val_loss: 0.0191
Epoch 79/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0194 - val_loss: 0.0183
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0043 - val_loss: 0.0029
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0042 - val_loss: 0.0029
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0041 - val_loss: 0.0028
Epoch 154/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0040 - val_loss: 0.0028
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0028
Epoch 156/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0039 - val_loss: 0.0027
Epoch 157/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0038 - val_loss: 0.0027
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 0.0027
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0036 - val_loss: 0.0026
Epoch 160/1000
32/32 [==============================] - 0s 188us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 9.7759e-04 - val_loss: 7.3077e-04
Epoch 231/1000
32/32 [==============================] - 0s 141us/step - loss: 9.6178e-04 - val_loss: 7.1647e-04
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 9.4632e-04 - val_loss: 7.0267e-04
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 9.3123e-04 - val_loss: 6.8931e-04
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 9.1648e-04 - val_loss: 6.7640e-04
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 9.0246e-04 - val_loss: 6.6380e-04
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 8.8812e-04 - val_loss: 6.5154e-04
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 8.7448e-04 - val_loss: 6.3962e-04
Epoch 238/1000
32/32 [==============================] - 0s 125us/step - loss: 8.6116e-04 - val_loss: 6.2

Epoch 304/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3636e-04 - val_loss: 3.0949e-04
Epoch 305/1000
32/32 [==============================] - 0s 125us/step - loss: 4.3511e-04 - val_loss: 3.0841e-04
Epoch 306/1000
32/32 [==============================] - 0s 155us/step - loss: 4.3388e-04 - val_loss: 3.0737e-04
Epoch 307/1000
32/32 [==============================] - 0s 187us/step - loss: 4.3265e-04 - val_loss: 3.0635e-04
Epoch 308/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3144e-04 - val_loss: 3.0535e-04
Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3024e-04 - val_loss: 3.0433e-04
Epoch 310/1000
32/32 [==============================] - 0s 186us/step - loss: 4.2906e-04 - val_loss: 3.0328e-04
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 4.2789e-04 - val_loss: 3.0222e-04
Epoch 312/1000
32/32 [==============================] - 0s 155us/step - loss: 4.2672e-04 - val_loss: 3.0

...

Epoch 748/1000
32/32 [==============================] - 0s 187us/step - loss: 2.7508e-04 - val_loss: 1.8469e-04
Epoch 749/1000
32/32 [==============================] - 0s 125us/step - loss: 2.7490e-04 - val_loss: 1.8442e-04
Epoch 750/1000
32/32 [==============================] - 0s 125us/step - loss: 2.7472e-04 - val_loss: 1.8421e-04
Epoch 751/1000
32/32 [==============================] - 0s 187us/step - loss: 2.7454e-04 - val_loss: 1.8402e-04
Epoch 752/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7436e-04 - val_loss: 1.8388e-04
Epoch 753/1000
32/32 [==============================] - 0s 125us/step - loss: 2.7418e-04 - val_loss: 1.8378e-04
Epoch 754/1000
32/32 [==============================] - 0s 187us/step - loss: 2.7400e-04 - val_loss: 1.8369e-04
Epoch 755/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7382e-04 - val_loss: 1.8361e-04
Epoch 756/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7364e-04 - val_loss: 1.8

Epoch 822/1000
32/32 [==============================] - 0s 155us/step - loss: 2.6298e-04 - val_loss: 1.7599e-04
Epoch 823/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6281e-04 - val_loss: 1.7626e-04
Epoch 824/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6265e-04 - val_loss: 1.7652e-04
Epoch 825/1000
32/32 [==============================] - 0s 155us/step - loss: 2.6249e-04 - val_loss: 1.7674e-04
Epoch 826/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6234e-04 - val_loss: 1.7693e-04
Epoch 827/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6220e-04 - val_loss: 1.7704e-04
Epoch 828/1000
32/32 [==============================] - 0s 187us/step - loss: 2.6205e-04 - val_loss: 1.7708e-04
Epoch 829/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6189e-04 - val_loss: 1.7703e-04
Epoch 830/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6173e-04 - val_loss: 1.7

Epoch 896/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5104e-04 - val_loss: 1.6742e-04
Epoch 897/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5087e-04 - val_loss: 1.6718e-04
Epoch 898/1000
32/32 [==============================] - 0s 124us/step - loss: 2.5070e-04 - val_loss: 1.6693e-04
Epoch 899/1000
32/32 [==============================] - 0s 155us/step - loss: 2.5053e-04 - val_loss: 1.6667e-04
Epoch 900/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5037e-04 - val_loss: 1.6641e-04
Epoch 901/1000
32/32 [==============================] - 0s 187us/step - loss: 2.5020e-04 - val_loss: 1.6617e-04
Epoch 902/1000
32/32 [==============================] - 0s 155us/step - loss: 2.5042e-04 - val_loss: 1.6589e-04
Epoch 903/1000
32/32 [==============================] - 0s 187us/step - loss: 2.4993e-04 - val_loss: 1.6569e-04
Epoch 904/1000
32/32 [==============================] - 0s 155us/step - loss: 2.4985e-04 - val_loss: 1.6

Epoch 970/1000
32/32 [==============================] - 0s 187us/step - loss: 2.3990e-04 - val_loss: 1.6011e-04
Epoch 971/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3974e-04 - val_loss: 1.5989e-04
Epoch 972/1000
32/32 [==============================] - 0s 125us/step - loss: 2.3958e-04 - val_loss: 1.5970e-04
Epoch 973/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3941e-04 - val_loss: 1.5956e-04
Epoch 974/1000
32/32 [==============================] - 0s 187us/step - loss: 2.3925e-04 - val_loss: 1.5947e-04
Epoch 975/1000
32/32 [==============================] - 0s 125us/step - loss: 2.3909e-04 - val_loss: 1.5940e-04
Epoch 976/1000
32/32 [==============================] - 0s 156us/step - loss: 2.3892e-04 - val_loss: 1.5935e-04
Epoch 977/1000
32/32 [==============================] - 0s 125us/step - loss: 2.3876e-04 - val_loss: 1.5930e-04
Epoch 978/1000
32/32 [==============================] - 0s 131us/step - loss: 2.3860e-04 - val_loss: 1.5

[array([[-0.05441995, -1.1184036 , -1.089987  , -0.04960766,  0.00361442],
        [-0.23498778,  0.19046225, -0.08051544,  0.26243466,  0.22370386],
        [-0.40012318,  0.689701  , -0.6838393 , -0.21574509,  0.8638539 ]],
       dtype=float32),
 array([ 0.6540333 ,  0.44112733,  0.2075105 , -0.8717559 ,  0.36762035],
       dtype=float32),
 array([[-0.64835525, -0.21112813,  0.18977189, -0.38953063,  0.7055551 ,
         -0.26851502,  0.30558392, -0.3648228 ,  0.6404661 , -0.26731387],
        [-0.7194904 , -0.5184312 ,  0.2555632 , -0.8974837 ,  0.1651725 ,
         -0.7520713 , -0.02376207,  0.5409103 ,  1.0019932 , -0.1418388 ],
        [-0.3140415 , -0.729762  , -0.21931088, -0.8262984 ,  0.6427943 ,
         -0.4043337 ,  0.5359972 ,  0.3964096 ,  0.13630104, -0.7974436 ],
        [-0.23275997, -1.255268  ,  0.39702213, -0.06795792,  0.18600374,
          0.6083439 , -0.06710818,  0.29643047,  0.26054835, -1.0486748 ],
        [-0.32302654,  0.2552393 ,  0.03333183, -0.3775265

In [51]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_relu_4th.h5')

In [52]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_sigmoid(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_29 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_44 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_30 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 10ms/step - loss: 54.11

Epoch 71/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0238 - val_loss: 0.0131
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0213 - val_loss: 0.0113
Epoch 73/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0188 - val_loss: 0.0096
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0163 - val_loss: 0.0079
Epoch 75/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0139 - val_loss: 0.0064
Epoch 76/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0117 - val_loss: 0.0051
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0097 - val_loss: 0.0040
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0079 - val_loss: 0.0031
Epoch 79/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0063 - val_loss: 0.0024
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 148/1000
32/32 [==============================] - 0s 125us/step - loss: 7.7401e-05 - val_loss: 3.2004e-04
Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 7.5060e-05 - val_loss: 3.2148e-04
Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 7.2972e-05 - val_loss: 3.2260e-04
Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 7.1100e-05 - val_loss: 3.2337e-04
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 6.9405e-05 - val_loss: 3.2375e-04
Epoch 153/1000
32/32 [==============================] - 0s 187us/step - loss: 6.7856e-05 - val_loss: 3.2377e-04
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 6.6428e-05 - val_loss: 3.2340e-04
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5089e-05 - val_loss: 3.2263e-04
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3819e-05 - val_loss: 3.2

Epoch 222/1000
32/32 [==============================] - 0s 187us/step - loss: 3.9084e-05 - val_loss: 2.3335e-04
Epoch 223/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9043e-05 - val_loss: 2.3303e-04
Epoch 224/1000
32/32 [==============================] - 0s 187us/step - loss: 3.9001e-05 - val_loss: 2.3271e-04
Epoch 225/1000
32/32 [==============================] - 0s 187us/step - loss: 3.8963e-05 - val_loss: 2.3239e-04
Epoch 226/1000
32/32 [==============================] - 0s 187us/step - loss: 3.8923e-05 - val_loss: 2.3208e-04
Epoch 227/1000
32/32 [==============================] - 0s 156us/step - loss: 3.8888e-05 - val_loss: 2.3175e-04
Epoch 228/1000
32/32 [==============================] - 0s 156us/step - loss: 3.8851e-05 - val_loss: 2.3144e-04
Epoch 229/1000
32/32 [==============================] - 0s 187us/step - loss: 3.8817e-05 - val_loss: 2.3113e-04
Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 3.8784e-05 - val_loss: 2.3

Epoch 296/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7435e-05 - val_loss: 2.2058e-04
Epoch 297/1000
32/32 [==============================] - 0s 187us/step - loss: 3.7418e-05 - val_loss: 2.2049e-04
Epoch 298/1000
32/32 [==============================] - 0s 125us/step - loss: 3.7402e-05 - val_loss: 2.2039e-04
Epoch 299/1000
32/32 [==============================] - 0s 125us/step - loss: 3.7385e-05 - val_loss: 2.2031e-04
Epoch 300/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7370e-05 - val_loss: 2.2021e-04
Epoch 301/1000
32/32 [==============================] - 0s 187us/step - loss: 3.7353e-05 - val_loss: 2.2012e-04
Epoch 302/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7337e-05 - val_loss: 2.2003e-04
Epoch 303/1000
32/32 [==============================] - 0s 187us/step - loss: 3.7320e-05 - val_loss: 2.1995e-04
Epoch 304/1000
32/32 [==============================] - 0s 156us/step - loss: 3.7306e-05 - val_loss: 2.1

...

Epoch 740/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1327e-05 - val_loss: 1.9021e-04
Epoch 741/1000
32/32 [==============================] - 0s 157us/step - loss: 3.1314e-05 - val_loss: 1.9015e-04
Epoch 742/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1302e-05 - val_loss: 1.9008e-04
Epoch 743/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1290e-05 - val_loss: 1.9001e-04
Epoch 744/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1278e-05 - val_loss: 1.8995e-04
Epoch 745/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1265e-05 - val_loss: 1.8989e-04
Epoch 746/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1253e-05 - val_loss: 1.8983e-04
Epoch 747/1000
32/32 [==============================] - 0s 125us/step - loss: 3.1240e-05 - val_loss: 1.8976e-04
Epoch 748/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1227e-05 - val_loss: 1.8

Epoch 814/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0429e-05 - val_loss: 1.8560e-04
Epoch 815/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0416e-05 - val_loss: 1.8553e-04
Epoch 816/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0404e-05 - val_loss: 1.8548e-04
Epoch 817/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0393e-05 - val_loss: 1.8542e-04
Epoch 818/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0381e-05 - val_loss: 1.8535e-04
Epoch 819/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0368e-05 - val_loss: 1.8529e-04
Epoch 820/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0357e-05 - val_loss: 1.8523e-04
Epoch 821/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0344e-05 - val_loss: 1.8517e-04
Epoch 822/1000
32/32 [==============================] - 0s 125us/step - loss: 3.0333e-05 - val_loss: 1.8

Epoch 888/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9555e-05 - val_loss: 1.8106e-04
Epoch 889/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9543e-05 - val_loss: 1.8101e-04
Epoch 890/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9532e-05 - val_loss: 1.8094e-04
Epoch 891/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9519e-05 - val_loss: 1.8089e-04
Epoch 892/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9508e-05 - val_loss: 1.8082e-04
Epoch 893/1000
32/32 [==============================] - 0s 374us/step - loss: 2.9497e-05 - val_loss: 1.8077e-04
Epoch 894/1000
32/32 [==============================] - 0s 218us/step - loss: 2.9485e-05 - val_loss: 1.8071e-04
Epoch 895/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9474e-05 - val_loss: 1.8064e-04
Epoch 896/1000
32/32 [==============================] - 0s 187us/step - loss: 2.9462e-05 - val_loss: 1.8

Epoch 962/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8707e-05 - val_loss: 1.7663e-04
Epoch 963/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8695e-05 - val_loss: 1.7656e-04
Epoch 964/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8684e-05 - val_loss: 1.7651e-04
Epoch 965/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8672e-05 - val_loss: 1.7645e-04
Epoch 966/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8662e-05 - val_loss: 1.7638e-04
Epoch 967/1000
32/32 [==============================] - 0s 218us/step - loss: 2.8649e-05 - val_loss: 1.7632e-04
Epoch 968/1000
32/32 [==============================] - 0s 125us/step - loss: 2.8638e-05 - val_loss: 1.7626e-04
Epoch 969/1000
32/32 [==============================] - 0s 125us/step - loss: 2.8626e-05 - val_loss: 1.7621e-04
Epoch 970/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8617e-05 - val_loss: 1.7

[array([[-1.7516611 , -1.0727636 ,  0.33554658, -1.7723533 , -1.4196906 ],
        [ 1.0386201 ,  0.2245127 ,  0.5973799 ,  1.1976243 ,  2.0291827 ],
        [-1.0725785 ,  0.47832027, -0.21666795, -0.4841909 , -1.2826316 ]],
       dtype=float32),
 array([ 1.1539018 , -0.08100377, -0.42295343,  1.2260271 ,  1.2702115 ],
       dtype=float32),
 array([[-0.71083605,  1.427217  , -1.2271223 ,  1.0533453 ,  1.133176  ,
         -1.3103006 ,  1.6028196 , -1.0257083 , -1.1024768 , -1.257393  ],
        [-0.27695006,  1.0491157 , -0.811491  ,  0.22327346,  0.8358492 ,
         -0.81335235,  0.88017726, -0.23114969, -0.28163013, -0.9056954 ],
        [-0.7298522 ,  1.3782674 , -0.22334653,  0.05625325,  0.78991735,
         -0.30199686,  0.14316753, -0.47533807, -0.77453405, -0.8926529 ],
        [-1.0017633 ,  0.84321386, -0.5846765 ,  1.1983545 ,  0.93464047,
         -1.4007223 ,  1.074967  , -0.9138205 , -0.761947  , -1.483029  ],
        [-0.75683326,  1.495155  , -1.4809438 ,  1.1769336

In [53]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_sigmoid_4th.h5')

In [54]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_tanh(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_31 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_47 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_32 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 10ms/step - loss: 38.22

Epoch 71/1000
32/32 [==============================] - 0s 155us/step - loss: 1.4930e-04 - val_loss: 2.3040e-04
Epoch 72/1000
32/32 [==============================] - 0s 155us/step - loss: 2.0974e-04 - val_loss: 7.5468e-04
Epoch 73/1000
32/32 [==============================] - 0s 187us/step - loss: 6.3124e-04 - val_loss: 0.0014
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0012 - val_loss: 0.0020
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0017 - val_loss: 0.0024
Epoch 76/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0021 - val_loss: 0.0025
Epoch 77/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0022 - val_loss: 0.0023
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0020 - val_loss: 0.0020
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0016 - val_loss: 0.0015
Epoch 80/1000
32/32 [==============================] 

Epoch 145/1000
32/32 [==============================] - 0s 156us/step - loss: 7.0516e-06 - val_loss: 5.2474e-06
Epoch 146/1000
32/32 [==============================] - 0s 156us/step - loss: 6.8390e-06 - val_loss: 5.9768e-06
Epoch 147/1000
32/32 [==============================] - 0s 156us/step - loss: 6.8335e-06 - val_loss: 6.7141e-06
Epoch 148/1000
32/32 [==============================] - 0s 123us/step - loss: 6.9723e-06 - val_loss: 7.3676e-06
Epoch 149/1000
32/32 [==============================] - 0s 155us/step - loss: 7.1790e-06 - val_loss: 7.8441e-06
Epoch 150/1000
32/32 [==============================] - 0s 155us/step - loss: 7.3724e-06 - val_loss: 8.0941e-06
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 7.4953e-06 - val_loss: 8.1018e-06
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 7.5161e-06 - val_loss: 7.8850e-06
Epoch 153/1000
32/32 [==============================] - 0s 187us/step - loss: 7.4385e-06 - val_loss: 7.4

Epoch 219/1000
32/32 [==============================] - 0s 155us/step - loss: 6.6590e-06 - val_loss: 4.9576e-06
Epoch 220/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6588e-06 - val_loss: 4.9725e-06
Epoch 221/1000
32/32 [==============================] - 0s 155us/step - loss: 6.6580e-06 - val_loss: 4.9833e-06
Epoch 222/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6567e-06 - val_loss: 4.9918e-06
Epoch 223/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6574e-06 - val_loss: 4.9967e-06
Epoch 224/1000
32/32 [==============================] - 0s 187us/step - loss: 6.6566e-06 - val_loss: 4.9992e-06
Epoch 225/1000
32/32 [==============================] - 0s 125us/step - loss: 6.6558e-06 - val_loss: 4.9989e-06
Epoch 226/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6552e-06 - val_loss: 4.9961e-06
Epoch 227/1000
32/32 [==============================] - 0s 187us/step - loss: 6.6545e-06 - val_loss: 4.9

Epoch 293/1000
32/32 [==============================] - 0s 125us/step - loss: 6.6172e-06 - val_loss: 4.9480e-06
Epoch 294/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6171e-06 - val_loss: 4.9485e-06
Epoch 295/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6158e-06 - val_loss: 4.9480e-06
Epoch 296/1000
32/32 [==============================] - 0s 125us/step - loss: 6.6159e-06 - val_loss: 4.9484e-06
Epoch 297/1000
32/32 [==============================] - 0s 125us/step - loss: 6.6150e-06 - val_loss: 4.9483e-06
Epoch 298/1000
32/32 [==============================] - 0s 155us/step - loss: 6.6143e-06 - val_loss: 4.9480e-06
Epoch 299/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6135e-06 - val_loss: 4.9473e-06
Epoch 300/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6134e-06 - val_loss: 4.9477e-06
Epoch 301/1000
32/32 [==============================] - 0s 125us/step - loss: 6.6129e-06 - val_loss: 4.9

...

Epoch 737/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3429e-06 - val_loss: 4.7890e-06
Epoch 738/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3428e-06 - val_loss: 4.7886e-06
Epoch 739/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3422e-06 - val_loss: 4.7881e-06
Epoch 740/1000
32/32 [==============================] - 0s 155us/step - loss: 6.3412e-06 - val_loss: 4.7874e-06
Epoch 741/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3407e-06 - val_loss: 4.7861e-06
Epoch 742/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3392e-06 - val_loss: 4.7869e-06
Epoch 743/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3389e-06 - val_loss: 4.7857e-06
Epoch 744/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3377e-06 - val_loss: 4.7863e-06
Epoch 745/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3375e-06 - val_loss: 4.7

Epoch 811/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2935e-06 - val_loss: 4.7591e-06
Epoch 812/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2927e-06 - val_loss: 4.7588e-06
Epoch 813/1000
32/32 [==============================] - 0s 125us/step - loss: 6.2915e-06 - val_loss: 4.7576e-06
Epoch 814/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2913e-06 - val_loss: 4.7587e-06
Epoch 815/1000
32/32 [==============================] - 0s 125us/step - loss: 6.2902e-06 - val_loss: 4.7595e-06
Epoch 816/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2899e-06 - val_loss: 4.7586e-06
Epoch 817/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2892e-06 - val_loss: 4.7565e-06
Epoch 818/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2882e-06 - val_loss: 4.7580e-06
Epoch 819/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2872e-06 - val_loss: 4.7

Epoch 885/1000
32/32 [==============================] - 0s 125us/step - loss: 6.2428e-06 - val_loss: 4.7280e-06
Epoch 886/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2415e-06 - val_loss: 4.7287e-06
Epoch 887/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2401e-06 - val_loss: 4.7287e-06
Epoch 888/1000
32/32 [==============================] - 0s 125us/step - loss: 6.2396e-06 - val_loss: 4.7280e-06
Epoch 889/1000
32/32 [==============================] - 0s 155us/step - loss: 6.2399e-06 - val_loss: 4.7276e-06
Epoch 890/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2387e-06 - val_loss: 4.7259e-06
Epoch 891/1000
32/32 [==============================] - 0s 156us/step - loss: 6.2382e-06 - val_loss: 4.7265e-06
Epoch 892/1000
32/32 [==============================] - 0s 125us/step - loss: 6.2372e-06 - val_loss: 4.7264e-06
Epoch 893/1000
32/32 [==============================] - 0s 155us/step - loss: 6.2367e-06 - val_loss: 4.7

Epoch 959/1000
32/32 [==============================] - 0s 155us/step - loss: 6.1902e-06 - val_loss: 4.6990e-06
Epoch 960/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1894e-06 - val_loss: 4.6977e-06
Epoch 961/1000
32/32 [==============================] - 0s 157us/step - loss: 6.1884e-06 - val_loss: 4.6983e-06
Epoch 962/1000
32/32 [==============================] - 0s 125us/step - loss: 6.1884e-06 - val_loss: 4.6978e-06
Epoch 963/1000
32/32 [==============================] - 0s 155us/step - loss: 6.1878e-06 - val_loss: 4.6971e-06
Epoch 964/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1874e-06 - val_loss: 4.6973e-06
Epoch 965/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1865e-06 - val_loss: 4.6974e-06
Epoch 966/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1856e-06 - val_loss: 4.6952e-06
Epoch 967/1000
32/32 [==============================] - 0s 157us/step - loss: 6.1843e-06 - val_loss: 4.6

[array([[-0.19875017, -0.02785967,  0.87802655, -0.47592303, -0.9999145 ],
        [ 1.3686749 ,  0.22447065, -0.17783053,  0.55345887,  1.0579501 ],
        [-1.6373907 ,  0.02080915,  1.1541448 , -1.5417757 ,  0.99988127]],
       dtype=float32),
 array([ 0.9039354 , -0.55581725,  0.8894419 ,  0.6240607 ,  0.8773968 ],
       dtype=float32),
 array([[-0.51839405,  1.3563112 , -0.86715317,  0.8473021 , -0.44729438,
         -0.83752525,  1.0757352 ,  1.037819  , -0.6472389 , -0.7182746 ],
        [-0.3013458 ,  0.74765426, -0.25244546,  0.8013971 , -0.6675782 ,
          0.1064247 ,  0.0852048 ,  0.4600611 , -0.40873614,  0.14464477],
        [-1.307476  ,  1.1251708 , -0.5906554 ,  0.62776226, -0.8181569 ,
          0.53651416,  1.0106112 ,  0.5936515 , -1.4444305 , -0.3708293 ],
        [-1.2465279 ,  0.4874122 , -0.8355048 ,  0.26276734, -1.3249125 ,
         -0.34477514,  0.8060374 ,  0.19794548, -1.2009432 ,  0.09766385],
        [-1.4315686 ,  0.37393025, -0.50850296,  1.1384532

In [55]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_tanh_4th.h5')